# Open-Sora 2.0 Training

## モデルの訓練

訓練は3つのステージで構成:

1. 低解像度動画によるT2V（Text to Video）モデルの訓練
1. 低解像度動画によるI2V（Image to Video）モデルの訓練
1. 高解像度動画によるI2Vモデルのファインチューニング

訓練のコストの内訳は、ステージ1のコストが50%を占める:

![](image/table3.png)

訓練の効率化の戦略:

- 11B（110億）パラメータのFluxの蒸留モデルで、T2Vモデルを初期化
    - Fluxは、オープンソースT2I（Text to Image）モデル
- 高品質な動画データで、訓練効率を向上
    - PixArtから着想
    - ステップ1, 2では、大規模データセットから高品質なサブセットを抽出
    - ステップ3では、より厳しいフィルタリングでサブセットを抽出
- 低解像度動画での動きの学習
    - 高解像度動画の訓練コストは高いため、多様な動きは低解像度動画を中心に訓練
        - 128フレーム768pxの動画の訓練は、256pxの場合よりも40倍遅い
        - セルフアテンションの計算複雑度が二次関数的に増大するため
    - 出力がぼやけるため、高解像度動画の訓練で品質を向上
- ステップ2のアップスケーリング学習は、T2VではなくI2Vを採用し効率化:
    - 静止画を条件付けすることで、動きの生成に集中できるため
- 高解像度動画でのファインチューニングを短縮し、訓練を効率化

Open-Sora 2.0の訓練コストは、5~10倍低い:

![](image/fig7.png)

訓練設定は、Open-Sora 1.2に基づく:

- 目的関数はFlow Matchingを採用
- オプティマイザはAdamWを使用
    - $\beta$値は$(0.9, 0.999)$
    - $\epsilon$は$1\times 10^{-15}$
    - 重み減衰は使用しない
    - 学習率
        - ステージ1, 2
            - 最初の4万ステップは$5\times 10^{-5}$
            - 最後の4.5万ステップは$3\times 10^{-5}$
        - ステージ3
            - $1\times 10^{-5}$
    - 購買ノルムクリッピングの閾値は$1$

Flow matchingは、Stable Diffusion 3（SD3）と類似:

$$
\mathcal{L} = \mathbb{E}_{t,X_{0},X_{1}}[||f_{\theta}(X_{t},t,y)-(X_{0}-X_{1})||]
$$

- $X_0$: 動画の潜在表現
- $X_1 \sim \mathcal{N}(0,1)$: ガウスノイズ
- $t$: タイムステップ（ノイズ強度）
- $X_t=(1-t)X_0 + tX_1$: 補間された潜在表現（ノイズありの潜在表現）
- $y$: テキストや画像などの条件
- $(X_0 - X_1)$: 正解の速度（データからノイズへの変化の方向）
- $\mathbb{E}_{t, X_0, X_1}$: 条件を変えた時の平均

$t$は、対数正規分布からサンプリングし、$X_0$の形状に合わせてスケーリング：

$$
t' = \frac{at}{1 + (a-1)t}
$$

- $a$: $T\times H\times W$に比例するパラメータ
- 高解像度での長時間の画像はノイズの影響を受けやすいため
- 推論時も同様の方法が適用

マルチバケットトレーニングを採用:

- フレーム数・解像度・アスペクト比が似た動画をバケツに分けて、バケツごとにバッチサイズを調整する手法
    - メモリ不足（OOM）の回避
    - 並列計算時の訓練時間の均一化

ステージ1, 2のバッチサイズとバッチサイズとスループット:

![](image/table4.png)

ステージ3のバッチサイズとスループット（Context parallelism 4）:

![](image/table5.png)

高圧縮オートエンコーダー（DC-AE）を使用し、トレーニングコストをさらに削減

DC-AEを使用すると、動画生成に必要なトークン数が大幅に削減できる:

$$
D_{token} = D_{T} \times D_{H} \times D_{W} \times P_{T} \times P_{H} \times P_{W}
$$

- $D_{\text{token}}$: トークン・ダウンサンプル比（どのくらい小さく圧縮してトークン化するかの比率）
- $D_T, D_H, H_W$: 時間・高さ・幅の次元におけるオートエンコーダの圧縮率
- $P_T, P_H, P_W$: 生成モデルに入力する際のパッチ分割サイズ
- Hunyuan Videoの$D_{\text{token}}$は$4096$、DC-AEの$D_{\text{token}}$は$1024$

オートエンコーダーの性能は、情報ダウンサンプリング比で予測できる:

$$
D_{info} = \frac{D_{T} \times D_{H} \times D_{W} \times C_{in}}{C_{out}}
$$

- $D_{\text{info}}$: 入力データが潜在表現になるときに情報量が圧縮された比率
- $C_{\text{in}}$: 入力チャンネル数（RGBの場合$3$）
- $C_{\text{out}}$: 出力チャンネル数（潜在表現のチャンネル数）
- DC-AEのオートエンコーダは、Hunyuan Video VAEとStepVideo VAEの$D_{\text{info}}$と一致するように設計
- 具体的には出力チャンネル数を増やす

DC-AEのダウンサンプルブロックとアップサンプルブロックには、勾配伝播の問題がある:

- ピクセルシャッフルとピクセルアンシャッフルを使用した残差接続を導入

DC-AEで出力した高圧縮の潜在変数でモデルを訓練すると、動画生成の品質が低くなる:

- チャンネル数が多いと、潜在空間構造（latent space structure）の最適化が難しくなるため
- DC-AE訓練後に、第3層の潜在表現をDINOv2に合わせるための蒸留損失を適用し、最適化

DC-AEを使用した動画生成モデルの訓練:

1. 2000万件の最大33フレームの動画を、1.7万ステップで訓練
2. 200万件の最大128フレームの動画を、8000ステップで訓練

DC-AEを使用した動画生成モデルは、Hunyuan Video VAEよりも、訓練と推論で高い効率性:

![](image/fig8.png)

DC-AEは、256チャネルと128チャネルの2つを最初から訓練:

1. 再構成損失$\mathcal{L}_1$と知覚損失$\mathcal{L}_{\text{LIPIS}}$を使用して、25万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L}_{1} + 0.5\mathcal{L}_{LPIPS}
    $$
2. 敵対的損失$\mathcal{L}_{\text{adv}}$を使用して、20万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L_1} + 0.5\mathcal{L_{LPIPS}} + 0.05\mathcal{L_{\text{adv}}}
    $$

DC-AEの訓練設定:

- ローカルバッチサイズ$1$で$8$個のGPUを使用
- アスペクト比$1:1$、32フレーム、256pxの動画を使用
- オプティマイザはAdamWを使用（$\beta=(0.9, 0.999)$, $\epsilon=1\times 10^{-15}$, 重み減衰なし）
- オートエンコーダの学習率は5e-5
- ディスクリミネータの学習率は1e-4
- 勾配ノルムクリップは閾値1

## 条件付け

条件付けは、潜在変数に対して条件ベクトルとタスクの種類を示すチャンネルを連結して行う:

![](image/fig10.png)

- 左: 画像から動画を生成するタスク
- 中: 動画を延長するタスク
- 右: 最初と最後の画像の間を補完するタスク
- 連結後のチャンネル数は$2k + 1$

汎化性能を向上させるために、テキスト条件のドロップアウトや画像条件のドロップアウトを導入

推論には、Classifier-free Guidance（CFG）を採用:

$$
v_{t} = v_{\theta}(x_{t},t,\emptyset,\emptyset) + g_{img}\cdot(v_{\theta}(x_{t},t,\emptyset,img)-v_{\theta}(x_{t},t,\emptyset,\emptyset)) + g_{txt}\cdot(v_{\theta}(x_{t},t,txt,img)-v_{\theta}(x_{t},t,\emptyset,img))
$$

- $g_{\text{img}}$: 画像条件の強さを調整する係数
- $g_{\text{txt}}$: テキスト条件の強さを調整する係数

画像ガイダンスを強くすると、生成された動画にチラつき（flicker）が生じる:

- Guidance Oscillationを導入して安定化
    - 例えば、50ステップのサンプリングのうち、10ステップごとに$g_{\text{img}}$を$1$に落とす

I2Vでは動的な画像ガイダンス（dynamic image guidance）を採用し、動画の後半になるにつれて条件付けを強くする:

![](image/fig11.png)

- 横軸はフレーム数
- 縦軸は推論ステップ数
- 画像のガイダンススケールは$1$から$k$まで線形にスケーリング
- 推論が終わりになるにつれて、ガイダンススケーリングが$1$になるように減衰
- ガイダンスのデフォルト値は$g_{\text{img}}=3$, $g_{\text{txt}}=7.5$

動画の動きの強さはモーションスコアをキャプションに追加することで制御:

![](image/fig12.png)

## システム最適化

訓練は、ColossalAIを使用:

- 141GBのメモリ容量を持つH200 GPUを使用し、データ並列（DP, Data Parallelism）を導入
- 選択的アクティベーションチェックポインティング（Selective Activation Checkpointing）を導入
    - 一部の順伝播の計算結果を破棄し、必要に応じて再計算する技術でメモリ効率化


高解像度の動画を効率的に処理するために、複数の並列化技術を採用:

- AEでは、テンソル並列化（TP, Tensor Parallelism）を畳み込み層に適用
    - 入力または出力のチャネル次元で重みを分割する
- MMDiTでは、Zero Redundancy Opitimizer（ZeroDP）とコンテキスト並列（CP, Context Parallelism）を適用
    - ZeroDPは、DeepSpeedのZeRO2により勾配を分割し、重複なくGPUが保持する技術
    - CPは、動画とテキストのシーケンスを分割し、各GPUが独立してアテンション計算する技術

アクティベーションチェックポインティングは、ブロックの入力のみを保持し、それ以外は再計算する:

- ステージ1, 2では、デュアルブロックの8層とすべてのシングルブロックに適用
- ステージ3では、すべてのブロックで有効化し、CPUへのオフロードも使用
    - オフロードでは、Pinned Memoryと非同期データ転送を使用し、さらに効率化

分散訓練の場合、物理障害が発生しやすいので自動復旧機能（Auto Recovery）を導入

- InfiniBandの障害、ストレージシステムのクラッシュ、NCCLエラー
- 訓練の状態を監視し、速度低下・損失の急激な悪化・レスポンスがないかを検証
- 問題が検出された場合は、すべてのプロセスを停止し、ノードを診断
- 必要に応じて予備をデプロイし、最終チェックポイントから自動的に訓練を再開する
- GPU稼働率99%で、ダウンタイムを最小限に抑えた

CPUとGPU間のデータ転送を高速化するためにPyTorchのデータローダーを最適化:

- PyTorchデフォルトのPinned Memoryは、CUDAカーネル実行をブロックする可能性がある
- メモリバッファ（pre-allocated pinned memory buffer）を使用し、オーバーヘッドを削減
- データ転送と計算をオーバーラップさせて効率化
- グローバルなGC（ガベージコレクション）は、分散訓練ではバグが多いため手動でメモリかんり

チェックポイントの保存は、ピン留めされたメモリバッファに対して直接アクセスすることで高速化:

- ディスクへの書き込みは、C++による非同期ディスク書き込みにより訓練プロセスをブロックせずに効率化

## 実装

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/hpcaitech/Open-Sora.git
    WORK_DIR = "/content/Open-Sora"
    %cd $WORK_DIR
except:
    IN_COLAB = False
    import os

    if not os.path.exists("/workspaces/open-sora-training/Open-Sora"):
        !git clone https://github.com/hpcaitech/Open-Sora.git
        !cp /workspaces/open-sora-training/ckpt.py /workspaces/open-sora-training/Open-Sora/opensora/utils/ckpt.py

    WORK_DIR = "/workspaces/open-sora-training/Open-Sora"
    %cd $WORK_DIR

In [ ]:
import logging
import os
import platform
import subprocess

DEBUG_LOG_PATH = os.path.join(WORK_DIR, "debug.log")

if os.path.exists(DEBUG_LOG_PATH):
    os.remove(DEBUG_LOG_PATH)

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

log_ = logging.getLogger()

for handler in log_.handlers:
    log_.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler(DEBUG_LOG_PATH)
file_handler.setFormatter(formatter)
log_.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
log_.addHandler(stream_handler)
log_.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
log_.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
log_.info(f"{NVIDIA_SMI}")

In [ ]:
if IN_COLAB:
    %pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

    %pip install \
        accelerate \
        av==13.1.0 \
        colossalai \
        ftfy \
        liger-kernel \
        omegaconf \
        mmengine \
        openai \
        pandas \
        pandarallel \
        pyarrow \
        tensorboard \
        wandb \
        --extra-index-url https://download.pytorch.org/whl/cu124

    %pip install flash-attn --no-build-isolation

    %pip install -e . --no-deps

In [ ]:
from collections import namedtuple
from collections import OrderedDict, defaultdict
from collections.abc import Iterable
from colossalai.booster import Booster
from colossalai.booster.plugin import HybridParallelPlugin, LowLevelZeroPlugin
from colossalai.cluster import DistCoordinator
from colossalai.utils import get_current_device
from colossalai.utils import set_seed
from colossalai.zero.low_level import LowLevelZeroOptimizer
from contextlib import nullcontext
from copy import deepcopy
from copy import deepcopy
from datetime import datetime
from datetime import timedelta
from einops import rearrange
from fractions import Fraction
from huggingface_hub import PyTorchModelHubMixin
from mmengine.config import Config
from pandarallel import pandarallel
from PIL import Image
from pprint import pformat
from torch import nn
from torch._utils import ExceptionWrapper
from torch.distributed import ProcessGroup
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.profiler import ProfilerActivity, profile, schedule
from torch.utils.checkpoint import (_DEFAULT_DETERMINISM_MODE, CheckpointFunction, _checkpoint_without_reentrant_generator, checkpoint_sequential, noop_context_fn)
from torch.utils.data import DataLoader, _utils
from torch.utils.data import Dataset, DistributedSampler
from torch.utils.data._utils import MP_STATUS_CHECK_INTERVAL
from torch.utils.data.dataloader import (IterDataPipe, MapDataPipe, _BaseDataLoaderIter, _MultiProcessingDataLoaderIter, _sharding_worker_init_fn, _SingleProcessDataLoaderIter)
from torch.utils.tensorboard import SummaryWriter
from torchvision import get_video_backend
from torchvision import models
from torchvision import models
from torchvision.datasets.folder import IMG_EXTENSIONS, pil_loader
from torchvision.io import write_video
from torchvision.io.video import _check_av_available
from torchvision.utils import save_image
from tqdm import tqdm
from typing import Any, Callable, Optional, Union, Tuple
from typing import Callable, ContextManager, Dict, List, Optional, Tuple
from typing import Iterator
import argparse
import ast
import av
import collections
import cv2
import diffusers
import functools
import gc
import hashlib
import json
import math
import matplotlib.pyplot as plt
import numbers
import numpy as np
import os
import pandas as pd
import queue
import random
import re
import requests
import subprocess
import threading
import torch
import torch.distributed as dist
import torch.multiprocessing as multiprocessing
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import wandb
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
gc.disable()

# 以前のワークショップで実装済み
from opensora.models.dc_ae.ae_model_zoo import DCAE_HF
from opensora.models.dc_ae.models.dc_ae import DCAE, DCAEConfig, dc_ae_f32
from opensora.utils.optimizer import create_lr_scheduler, create_optimizer

## 動画データのダウンロード

動画ファイルは[hcpai-tech/open-sora-pexels-45k][1]にある

[1]: https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k

In [ ]:
# 269.53GBの動画データセットをダウンロード
# https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k

if False:
    !apt update && apt install git-lfs
    !git-lfs install
    !git clone "https://www.modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k.git"
    !cd open-sora-pexels-45k && \
        cat tar/pexels_45k.tar.* > pexels_45k.tar && \
        mkdir ../datasets && \
        mv pexels_45k ../datasets

重いので最小限のCSVデータのみをダウンロード

In [ ]:
!wget -nc https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k/resolve/master/pexels_45k_necessary.csv

In [ ]:
pexels_45k_necessary_df = pd.read_csv("pexels_45k_necessary.csv")
print(len(pexels_45k_necessary_df)) # 45,817件
pexels_45k_necessary_df.head(5)

In [ ]:
pexels_45k_score_df = pexels_45k_necessary_df.sort_values(by=["num_frames"])
pexels_5 = pexels_45k_score_df.head(5).copy()
pexels_5['path'] = pexels_5['path'].str.replace('datasets/pexels_45k/', 'datasets/pexels_5/', regex=False)
pexels_5

In [ ]:
DATASETS_DIR = os.path.join(WORK_DIR, "datasets")
PEXELS_DATASET_DIR = os.path.join(DATASETS_DIR, "pexels_5")
os.makedirs(PEXELS_DATASET_DIR, exist_ok=True)

In [ ]:
PEXELS_CSV_PATH = os.path.join(WORK_DIR, "pexels_5.csv")

if not os.path.exists(PEXELS_CSV_PATH):
    for path in pexels_5["path"]:
        match = re.search(r"/(\d+)_", path)
        video_id = match.group(1)
        video_url = f"https://www.pexels.com/download/video/{video_id}"
        video_path = os.path.join(PEXELS_DATASET_DIR, f"{video_id}.mp4")
        !wget -nc -O $video_path $video_url
        pexels_5["path"] = pexels_5["path"].replace(path, video_path)

    pexels_5.to_csv(os.path.join(WORK_DIR, "pexels_5.csv"), index=False)
else:
    pexels_5 = pd.read_csv(PEXELS_CSV_PATH)

pexels_5.head(5)

## DC-AEの訓練

[ドキュメント][1]より以下のコマンドで開始できる

```sh
torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae.py
```

[1]: https://github.com/hpcaitech/Open-Sora/blob/main/docs/hcae.md

### CLIパーサ

In [ ]:
def parse_args(args) -> tuple[str, argparse.Namespace]:
    """
    コマンドラインの引数をパースする
    parse_configs関数で使用する

    Args:
        args (list[str]): コマンドライン引数のリスト
    Returns:
        tuple[str, argparse.Namespace]: 設定ファイルのパスとコマンドライン引数
    """
    # argparseを初期化
    parser = argparse.ArgumentParser()

    # config引数を追加
    parser.add_argument("config", type=str, help="model config file path")

    # 既知の引数をパースし、未知の引数をunknown_argsとして取得
    args, unknown_args = parser.parse_known_args(args)

    return args.config, unknown_args

# 検証
parse_args(["train.py", "-m", "dcae", "-i", "12345"])

In [ ]:
def read_config(config_path: str) -> Config:
    """
    設定ファイルを読み込む
    parse_configs関数で使用する

    Args:
        config_path (str): 設定ファイルのパス
    Returns:
        Config: 設定オブジェクト
    """

    # mmengineのConfigクラスを使用して設定ファイルを読み込む
    # configs/vae/train/video_dc_ae.py
    # https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#read-the-configuration-file
    cfg = Config.fromfile(config_path)

    return cfg

# 検証
read_config("configs/vae/train/video_dc_ae.py")

In [ ]:
def auto_convert(value: str) -> int | float | bool | list | dict | None:
    """
    文字列を適切なPythonデータ型に自動変換する
    int、float、bool、list、dictなどを含む
    merge_args関数で使用する

    Args:
        value (str): 変換する文字列
    Returns:
        int, float, bool, list |  dict: 変換後の値
    """

    # 空白文字はそのまま
    if value == "":
        return value

    # noneはNoneに変換
    if value.lower() == "none":
        return None

    # boolean値を処理
    lower_value = value.lower()
    if lower_value == "true":
        return True
    elif lower_value == "false":
        return False

    # 文字列をintまたはfloatに変換を試みる
    try:
        # Integerに変換
        return int(value)
    except ValueError:
        pass

    try:
        # Floatに変換
        return float(value)
    except ValueError:
        pass

    # 文字列をlist、dict、tupleなどに変換を試みる
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        pass

    # そのまま文字列を返す
    return value

# 検証
auto_convert(""), auto_convert("true"), auto_convert("123"), auto_convert("1.23")

In [ ]:
def merge_args(cfg: Config, args: argparse.Namespace) -> Config:
    """
    設定ファイルとコマンドライン引数をマージする関数

    Args:
        cfg (Config): 設定オブジェクト
        args (argparse.Namespace): コマンドライン引数
    Returns:
        Config: マージ後の設定オブジェクト
    """

    # 引数のオプション（偶数番目）と値（奇数番目）をペアにして処理
    for k, v in zip(args[::2], args[1::2]):

        # 1) 検証と前処理

        assert k.startswith("--"), f"Invalid argument: {k}"

        # --foo-bar -> foo_bar
        k = k[2:].replace("-", "_")

        # foo.bar -> ["foo", "bar"]
        k_split = k.split(".")

        # 2) 置き換えるキーを含む階層の参照を取得

        target = cfg
        for key in k_split[:-1]:
            assert key in cfg, f"Key {key} not found in config"
            target = target[key]

        # 3) 置き換える値の型を推測して変換
            
        if v.lower() == "none":
            v = None
        elif k in target:
            v_type = type(target[k])
            if v_type == bool:
                v = auto_convert(v)
            else:
                v = type(target[k])(v)
        else:
            v = auto_convert(v)

        # 4) 値を置き換え
        target[k_split[-1]] = v

    return cfg

# 検証
merge_args(read_config("configs/vae/train/video_dc_ae.py"), ['--model.type', 'dc_ae'])

In [ ]:
def parse_configs(args) -> Config:
    """
    設定ファイルとコマンドライン引数をパースする

    Args:
        args (list[str]): コマンドライン引数のリスト
    Returns:
        Config: 設定オブジェクト
    """

    # コマンドラインの引数をパース
    config, args = parse_args(args)

    # 設定ファイルを読み込む
    cfg = read_config(config)

    # 設定ファイルとコマンドライン引数をマージ
    cfg = merge_args(cfg, args)

    # 設定ファイルのパスを保存
    cfg.config_path = config

    # 環境変数AE_SPATIAL_COMPRESSIONを設定
    if cfg.get("ae_spatial_compression", None) is not None:
        os.environ["AE_SPATIAL_COMPRESSION"] = str(cfg.ae_spatial_compression)

    return cfg

cfg = parse_configs(["configs/vae/train/video_dc_ae.py"])
cfg

In [ ]:
# ワークショップ用に値を上書き

cfg.bucket_config = {
    "256px_ar1:1": {
        4: (1.0, 1), # 4フレームの動画を1バッチ
    }
}

cfg.epochs = 1 # 100から1に変更

cfg.num_workers = 0
cfg.prefetch_factor = None
cfg.pin_memory_cache_pre_alloc_numels = None 

# cfg.accumulation_steps = 128

### データ型設定

In [ ]:
def to_torch_dtype(dtype: str | torch.dtype) -> torch.dtype:
    """
    文字列をtorch.dtypeに変換する

    Args:
        dtype (str | torch.dtype): 変換する文字列またはtorch.dtype
    Returns:
        torch.dtype: 変換後のtorch.dtype
    """

    # すでにtorch.dtypeの場合
    # False
    if isinstance(dtype, torch.dtype):
        return dtype

    # 文字列の場合
    # True
    elif isinstance(dtype, str):

        # 文字列をtorch.dtypeにマッピング
        dtype_mapping = {
            "float64": torch.float64,
            "float32": torch.float32,
            "float16": torch.float16,
            "fp32": torch.float32,
            "fp16": torch.float16,
            "half": torch.float16,
            "bf16": torch.bfloat16,
        }
        if dtype not in dtype_mapping:
            raise ValueError(f"Unsupported dtype {dtype}")

        # 対応するtorch.dtypeを返す
        dtype = dtype_mapping[dtype]
        return dtype
    
    # False
    else:
        raise ValueError(f"Unsupported dtype {dtype}")

dtype = to_torch_dtype(cfg.get("dtype", "bf16"))
dtype

### デバイス設定

In [ ]:
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

In [ ]:
def setup_device() -> tuple[torch.device, DistCoordinator]:
    """
    デバイスと分散コーディネーターをセットアップする

    Returns:
        tuple[torch.device, DistCoordinator]: デバイスと分散コーディネーター
    """

    assert torch.cuda.is_available(), "Training currently requires at least one GPU."

    # NCCLバックエンドで分散プロセスグループを初期化
    # NCCL = NVIDIA Collective Communications Library
    # NOTE: プロセスの早期終了を防ぐために非常に大きなタイムアウトを設定
    dist.init_process_group(backend="nccl", timeout=timedelta(hours=24))

    # 「現在のプロセスのランクID % 利用可能なGPU数」を計算してデバイスを設定
    # 例えば、ランクIDが3でGPUが2台の場合、デバイスは1になる
    torch.cuda.set_device(dist.get_rank() % torch.cuda.device_count())

    # Colossal-AIのコーディネーターを初期化
    # https://colossalai.org/docs/features/cluster_utils/#api-reference
    coordinator = DistCoordinator()

    # 現在のデバイスを取得
    device = get_current_device()

    return device, coordinator

try:
    device, coordinator = setup_device()
except Exception as e:
    print(e)
    pass

device, coordinator

### チェックポイントモジュール

In [ ]:
class CheckpointIO:
    """
    チェックポイントの読み込みと書き込みを管理するクラス
    """

    def __init__(self, n_write_entries: int = 32):
        # 書き込みエントリ数の上限
        self.n_write_entries = n_write_entries

        # 書き込みオブジェクト
        self.writer: Optional[Any] = None

        # ピン留めされた状態辞書
        self.pinned_state_dict: Optional[Dict[str, torch.Tensor]] = None

        # マスターのピン留めされた状態辞書
        # マスターは、分散学習の主ノード
        self.master_pinned_state_dict: Optional[Dict[str, torch.Tensor]] = None

        # マスターの書き込みオブジェクト
        self.master_writer: Optional[Any] = None

    def _sync_io(self):
        # 書き込みオブジェクトが存在する場合、同期して解放
        if self.writer is not None:
            self.writer.synchronize()
            self.writer = None

        # マスターの書き込みオブジェクトが存在する場合、同期して解放
        if self.master_writer is not None:
            self.master_writer.synchronize()
            self.master_writer = None

    def __del__(self):
        self._sync_io()

    def _prepare_pinned_state_dict(self, ema: nn.Module, ema_shape_dict: dict):
        if self.pinned_state_dict is None and dist.get_rank() == 0:
            self.pinned_state_dict = _prepare_ema_pinned_state_dict(ema, ema_shape_dict)

    def _prepare_master_pinned_state_dict(self, model: nn.Module, optimizer: LowLevelZeroOptimizer):
        if self.master_pinned_state_dict is None and dist.get_rank() == 0:
            sd = {}
            w2m = optimizer.get_working_to_master_map()
            for n, p in model.named_parameters():
                master_p = w2m[id(p)]
                sd[n] = torch.empty(p.shape, dtype=master_p.dtype, pin_memory=True, device="cpu")
            self.master_pinned_state_dict = sd

    def save(
        self,
        booster: Booster,
        save_dir: str,
        model: nn.Module = None,
        ema: nn.Module = None,
        optimizer: Optimizer = None,
        lr_scheduler: _LRScheduler = None,
        sampler=None,
        epoch: int = None,
        step: int = None,
        global_step: int = None,
        batch_size: int = None,
        lora: bool = False,
        actual_update_step: int = None,
        ema_shape_dict: dict = None,
        async_io: bool = True,
        include_master_weights: bool = False,
    ) -> str:
        """
        チェックポイントを保存する

        Args:
            booster (Booster): Boosterオブジェクト
            save_dir (str): チェックポイントを保存するディレクトリ
            model (nn.Module): チェックポイントを保存するモデル
            ema (nn.Module): チェックポイントを保存するEMAモデル
            optimizer (Optimizer): チェックポイントを保存するオプティマイザ
            lr_scheduler (_LRScheduler): チェックポイントを保存する学習率スケジューラ
            sampler: チェックポイントを保存するサンプラー
            epoch (int): チェックポイントのエポック
            step (int): チェックポイントのステップ
            global_step (int): チェックポイントのグローバルステップ
            batch_size (int): チェックポイントのバッチサイズ
            lora (bool): モデルがLoRAで訓練されているかどうか
            actual_update_step (int): 実際の更新ステップ
            ema_shape_dict (dict): EMAモデルの形状辞書
            async_io (bool): 非同期I/Oを使用するかどうか
            include_master_weights (bool): マスターウェイトを含めるかどうか

        Returns:
            str: 保存されたチェックポイントのパス
        """

        # 書き込みオブジェクトが存在する場合、同期し解放
        self._sync_io()

        # 保存ディレクトリパスを作成
        save_dir = os.path.join(save_dir, f"epoch{epoch}-global_step{actual_update_step}")

        # 非同期I/Oのデバッグログパスを設定
        os.environ["TENSORNVME_DEBUG_LOG"] = os.path.join(save_dir, "async_file_io.log")

        # モデルが存在する場合
        if model is not None:

            # LoRAでない場合
            if not lora:
                # モデルディレクトリを作成
                os.makedirs(os.path.join(save_dir, "model"), exist_ok=True)

                # Colossal-AIのBoosterを使用してモデルを保存
                # https://colossalai.org/docs/basics/booster_api/
                # NOTE: プロセスごとに独立して処理
                booster.save_model(
                    model,
                    os.path.join(save_dir, "model"),
                    shard=True, # シャードに分割して保存
                    use_safetensors=True, # safetensors形式で保存
                    size_per_shard=4096, # シャードあたりのサイズを4096MBに設定
                    use_async=async_io, # 非同期I/Oを使用
                )

            # LoRAの場合
            else:
                # LoRAディレクトリを作成
                os.makedirs(os.path.join(save_dir, "lora"), exist_ok=True)

                # LoRAモデルを保存
                booster.save_lora_as_pretrained(model, os.path.join(save_dir, "lora"))

        # オプティマイザが存在する場合
        if optimizer is not None:

            # オプティマイザを保存
            booster.save_optimizer(
                optimizer,
                os.path.join(save_dir, "optimizer"),
                shard=True,
                size_per_shard=4096,
                use_async=async_io
            )

            # マスターウェイトを含める場合
            # マスターウェイトは、高精度で分散して保存されているウェイト
            if include_master_weights:

                # マスターのピン留めされた状態辞書を準備
                self._prepare_master_pinned_state_dict(model, optimizer)

                # マスターウェイトを収集
                master_weights_gathering(model, optimizer, self.master_pinned_state_dict)

        # 学習率スケジューラが存在する場合
        if lr_scheduler is not None:

            # 学習率スケジューラを保存
            booster.save_lr_scheduler(lr_scheduler, os.path.join(save_dir, "lr_scheduler"))

        # EMAモデルが存在する場合
        # EMA = Exponential Moving Average（移動平均）
        if ema is not None:

            # EMAモデルのピン留めされた状態辞書を準備
            self._prepare_pinned_state_dict(ema, ema_shape_dict)

            # EMAモデルの状態を収集
            model_gathering(ema, ema_shape_dict, self.pinned_state_dict)

        # ランク0のプロセスのみが実行
        if dist.get_rank() == 0:

            # 訓練の進行状況の辞書を作成
            running_states = {
                "epoch": epoch,
                "step": step,
                "global_step": global_step,
                "batch_size": batch_size,
                "actual_update_step": actual_update_step,
            }

            # 進行状況をJSONファイルとして保存
            save_json(running_states, os.path.join(save_dir, "running_states.json"))

            # EMAモデルが存在する場合
            if ema is not None:

                # 非同期I/Oの場合
                if async_io:
                    # safetensors形式でEMAモデルを非同期保存
                    self.writer = async_save(os.path.join(save_dir, "ema.safetensors"), self.pinned_state_dict)

                # 通常I/Oの場合
                else:
                    # EMAモデルを通常のPyTorch形式で保存
                    torch.save(ema.state_dict(), os.path.join(save_dir, "ema.pt"))

            # サンプラーが存在する場合
            if sampler is not None:
                # サンプラーの状態を保存
                # （現在はVariableVideoBatchSamplerのみ対応）
                torch.save(sampler.state_dict(step), os.path.join(save_dir, "sampler"))

            # マスターウェイトが存在する場合
            if optimizer is not None and include_master_weights:

                # マスターウェイトを非同期保存
                self.master_writer = async_save(
                    os.path.join(save_dir, "master.safetensors"), self.master_pinned_state_dict
                )

        # すべてのプロセスが完了するまで待機
        dist.barrier()

        return save_dir

    def load(
        self,
        booster: Booster,
        load_dir: str,
        model: nn.Module = None,
        ema: nn.Module = None,
        optimizer: Optimizer = None,
        lr_scheduler: _LRScheduler = None,
        sampler=None,
        strict: bool = False,
        include_master_weights: bool = False,
    ) -> tuple[int, int]:
        """
        チェックポイントを読み込む

        Args:
            booster (Booster): Boosterオブジェクト
            load_dir (str): チェックポイントを読み込むディレクトリ
            model (nn.Module): チェックポイントを読み込むモデル
            ema (nn.Module): チェックポイントを読み込むEMAモデル
            optimizer (Optimizer): チェックポイントを読み込むオプティマイザ
            lr_scheduler (_LRScheduler): チェックポイントを読み込む学習率スケジューラ
            sampler: チェックポイントを読み込むサンプラー
            strict (bool): 厳密に読み込むかどうか
            include_master_weights (bool): マスターウェイトを含めるかどうか
        Returns:
            tuple[int, int]: チェックポイントのエポックとステップ
        """

        # ディレクトリの存在を確認
        assert os.path.exists(load_dir), f"Checkpoint directory {load_dir} does not exist"
        assert os.path.exists(os.path.join(load_dir, "running_states.json")), "running_states.json does not exist"

        # 訓練の進行状況を読み込む
        running_states = load_json(os.path.join(load_dir, "running_states.json"))

        # モデルが存在する場合
        if model is not None:

            # モデルを読み込む
            booster.load_model(
                model,
                _search_valid_path(os.path.join(load_dir, "model")),
                strict=strict, # キーが厳密に一致するかどうか
                low_cpu_mem_mode=False, # CPUメモリを節約するモード
                num_threads=32, # 読み込みに使用するスレッド数
            )

        # EMAモデルが存在する場合
        if ema is not None:
            # safetensors形式の場合
            if os.path.exists(os.path.join(load_dir, "ema.safetensors")):
                ema_state_dict = load_file(os.path.join(load_dir, "ema.safetensors"))

            # 通常のPyTorch形式の場合
            else:
                ema_state_dict = torch.load(os.path.join(load_dir, "ema.pt"), map_location=torch.device("cpu"))

            # EMAはBoosterで強化されていないため、booster.load_modelは使用しない
            ema.load_state_dict(ema_state_dict, strict=strict, assign=True)

        # オプティマイザが存在する場合
        if optimizer is not None:

            # オプティマイザを読み込む
            booster.load_optimizer(
                optimizer, os.path.join(load_dir, "optimizer"), low_cpu_mem_mode=False, num_threads=32
            )

            # マスターウェイトを含める場合
            if include_master_weights:
                master_state_dict = load_file(os.path.join(load_dir, "master.safetensors"))
                # オプティマイザにマスターウェイトを読み込む
                load_master_weights(model, optimizer, master_state_dict)

        # 学習率スケジューラが存在する場合
        if lr_scheduler is not None:
            # 学習率スケジューラを読み込む
            booster.load_lr_scheduler(lr_scheduler, os.path.join(load_dir, "lr_scheduler"))

        # サンプラーが存在する場合
        if sampler is not None:
            # サンプラーの状態を読み込む
            sampler.load_state_dict(torch.load(os.path.join(load_dir, "sampler")))

        # すべてのプロセスが完了するまで待機
        dist.barrier()

        return (running_states["epoch"], running_states["step"])

checkpoint_io = CheckpointIO()

### シード固定

In [ ]:
set_seed(cfg.get("seed", 1024))

### ピンメモリー設定

In [ ]:
import torch
tensor = torch.randn(3, 3)
id(tensor)

In [ ]:
class PinMemoryCache:
    """
    ピンメモリを再利用するキャッシュクラス
    ピンメモリは、CPUメモリ上でページングされないメモリ領域（ピン留めされたメモリ）
    ピンメモリを使用すると、CPUにアドレスを問い合わせずにGPUにデータを転送できる
    """

    # キャッシュするテンソルのデータ型を矯正する
    force_dtype: Optional[torch.dtype] = None

    # キャッシュするテンソルの最小要素数
    min_cache_numel: int = 0

    # 事前に確保するテンソルの要素数リスト
    pre_alloc_numels: List[int] = []

    def __init__(self):

        # IDをキーにしたテンソルキャッシュ
        self.cache: Dict[int, torch.Tensor] = {}

        # 
        self.output_to_cache: Dict[int, int] = {}

        # 
        self.cache_to_output: Dict[int, int] = {}

        # スレッド安全のためのロック
        self.lock = threading.Lock()
        self.total_cnt = 0
        self.hit_cnt = 0

        if len(self.pre_alloc_numels) > 0 and self.force_dtype is not None:
            for n in self.pre_alloc_numels:
                cache_tensor = torch.empty(n, dtype=self.force_dtype, device="cpu", pin_memory=True)
                with self.lock:
                    self.cache[id(cache_tensor)] = cache_tensor

    def get(self, tensor: torch.Tensor) -> torch.Tensor:
        """
        CPUテンソルを受け取り、対応するピン留めされたテンソルを返す
        メモリの割り当てのみを管理し、内容はコピーしない

        Args:
            tensor (torch.Tensor): ピン留めするテンソル
        Returns:
            torch.Tensor: ピン留めされたテンソル
        """

        # 呼び出し回数をインクリメント
        self.total_cnt += 1

        # ロックを取得
        with self.lock:

            # キャッシュのIDと値でループ
            for cache_id, cache_tensor in self.cache.items():

                # キャッシュが未使用で、サイズが十分ある場合
                if cache_id not in self.cache_to_output and cache_tensor.numel() >= tensor.numel():

                    # 対応するキャッシュテンソルを作成
                    target_cache_tensor = cache_tensor[: tensor.numel()].view(tensor.shape)

                    # 出力テンソルのオブジェクトIDを取得
                    # NOTE: オブジェクト生存中のみIDは一意
                    out_id = id(target_cache_tensor)

                    # 出力テンソルとキャッシュテンソルのマッピングを保存
                    self.output_to_cache[out_id] = cache_id

                    # キャッシュテンソルと出力テンソルのマッピングを保存
                    self.cache_to_output[cache_id] = out_id

                    # ヒット回数をインクリメント
                    self.hit_cnt += 1

                    # 対応するキャッシュテンソルを返す
                    return target_cache_tensor

        # データ型を取得
        dtype = self.force_dtype if self.force_dtype is not None else tensor.dtype

        # キャッシュするテンソルの要素数を決定
        cache_numel = max(tensor.numel(), self.min_cache_numel)

        # 新しいキャッシュテンソルを作成
        # NOTE: pin_memory=Trueでピンメモリに配置
        cache_tensor = torch.empty(cache_numel, dtype=dtype, device="cpu", pin_memory=True)

        # 入力のテンソル形状に合わせてキャッシュテンソルを整形
        target_cache_tensor = cache_tensor[: tensor.numel()].view(tensor.shape)

        # キャッシュテンソルのオブジェクトIDを取得
        out_id = id(target_cache_tensor)

        # ロックを取得
        with self.lock:

            # キャッシュに新しいキャッシュテンソルを追加
            self.cache[id(cache_tensor)] = cache_tensor

            # 出力テンソルとキャッシュテンソルのマッピングを保存
            self.output_to_cache[out_id] = id(cache_tensor)

            # キャッシュテンソルと出力テンソルのマッピングを保存
            self.cache_to_output[id(cache_tensor)] = out_id

        # 対応するキャッシュテンソルを返す
        return target_cache_tensor

    def remove(self, output_tensor: torch.Tensor) -> None:
        """Release corresponding cache tensor.

        Args:
            output_tensor (torch.Tensor): The tensor to be released.
        """
        out_id = id(output_tensor)
        with self.lock:
            if out_id not in self.output_to_cache:
                raise ValueError("Tensor not found in cache.")
            cache_id = self.output_to_cache.pop(out_id)
            del self.cache_to_output[cache_id]

    def __str__(self):
        with self.lock:
            num_cached = len(self.cache)
            num_used = len(self.output_to_cache)
            total_cache_size = sum([v.numel() * v.element_size() for v in self.cache.values()])
        return f"PinMemoryCache(num_cached={num_cached}, num_used={num_used}, total_cache_size={total_cache_size / 1024**3:.2f} GB, hit rate={self.hit_cnt / self.total_cnt:.2f})"

PinMemoryCache.force_dtype = dtype
pin_memory_cache_pre_alloc_numels = cfg.get("pin_memory_cache_pre_alloc_numels", None)
PinMemoryCache.pre_alloc_numels = pin_memory_cache_pre_alloc_numels

### CollosalAI Boosterの初期化

In [ ]:
# グローバルにアクセスできるデータ並列のプロセスグループの辞書を作成
# プロセスグループは、分散学習で通信・同期するプロセスの集合
_GLOBAL_PARALLEL_GROUPS = dict()

def set_data_parallel_group(group: dist.ProcessGroup):
    """
    データ並列のプロセスグループをグローバルに設定する

    Args:
        group (dist.ProcessGroup): データ並列のプロセスグループ
    """
    _GLOBAL_PARALLEL_GROUPS["data"] = group

In [ ]:
def create_colossalai_plugin(
    plugin: str,
    dtype: str,
    grad_clip: float,
    **kwargs,
) -> LowLevelZeroPlugin | HybridParallelPlugin:
    """
    ColossalAIの並列プラグインを作成する
    並列化の設定を管理する重要なクラス
    Zero1、Zero2、HybridParallelのいずれかを選択可能

    Args:
        plugin (str): プラグイン名
        dtype (str): データ型
        grad_clip (float): 勾配クリップ値
    Returns:
        LowLevelZeroPlugin | HybridParallelPlugin: プラグイン
    """
    # プラグインの引数を設定
    plugin_kwargs = dict(
        precision=dtype, # bf16
        initial_scale=2**16,
        max_norm=grad_clip, # 1.0
        overlap_allgather=True,
        cast_inputs=False,
        reduce_bucket_size_in_m=20,
    )

    # キーワード引数で上書き
    plugin_kwargs.update(kwargs)

    # sequence parallelismのサイズを取得
    # シーケンス並列は、シーケンスデータを複数のGPUに分割して処理する手法
    sp_size = plugin_kwargs.get("sp_size", 1)

    # True
    if plugin == "zero1" or plugin == "zero2":
        assert sp_size == 1, "Zero plugin does not support sequence parallelism"

        # zero2 -> 2
        stage = 1 if plugin == "zero1" else 2

        # LowLevelZeroPluginを作成
        # https://colossalai.org/docs/basics/booster_plugins/#low-level-zero-plugin
        plugin = LowLevelZeroPlugin(
            stage=stage, # 2
            **plugin_kwargs,
        )

        # グローバルにデータ並列のプロセスグループを設定
        set_data_parallel_group(dist.group.WORLD)

    # False
    elif plugin == "hybrid":
        plugin_kwargs["find_unused_parameters"] = True
        reduce_bucket_size_in_m = plugin_kwargs.pop("reduce_bucket_size_in_m")
        if "zero_bucket_size_in_m" not in plugin_kwargs:
            plugin_kwargs["zero_bucket_size_in_m"] = reduce_bucket_size_in_m
        plugin_kwargs.pop("cast_inputs")
        plugin_kwargs["enable_metadata_cache"] = False

        custom_policy = plugin_kwargs.pop("custom_policy", None)
        if custom_policy is not None:
            custom_policy = custom_policy()
        plugin = HybridParallelPlugin(
            custom_policy=custom_policy,
            **plugin_kwargs,
        )
        set_tensor_parallel_group(plugin.tp_group)
        set_sequence_parallel_group(plugin.sp_group)
        set_data_parallel_group(plugin.dp_group)

    # False
    else:
        raise ValueError(f"Unknown plugin {plugin}")
    return plugin

# zero2
plugin_type = cfg.get("plugin", "zero2")

# dtype=bf16, grad_clip=1.0
plugin_config = cfg.get("plugin_config", {})

# プラグインを作成
plugin = (
    create_colossalai_plugin(
        plugin=plugin_type,
        dtype=cfg.get("dtype", "bf16"),
        grad_clip=cfg.get("grad_clip", 0),
        **plugin_config,
    )
    if plugin_type != "none"
    else None
)

plugin

In [ ]:
# プラグインを指定してBoosterを初期化
# Boosterは、モデル・オプティマイザ・データローダに対し並列化機能を注入するメインモジュール
# https://colossalai.org/docs/basics/booster_api
booster = Booster(plugin=plugin)

### 実験用ディレクトリの作成

In [ ]:
def is_distributed() -> bool:
    """
    分散設定かどうかを判定する

    Returns:
        bool: 分散設定の場合はTrue、そうでない場合はFalse
    """

    # WORLD_SIZE環境変数が設定されているかどうかで判定
    return os.environ.get("WORLD_SIZE", None) is not None

# 検証
is_distributed()

In [ ]:
def is_main_process() -> bool:
    """
    現在のプロセスがメインプロセスかどうかを判定する

    Returns:
        bool: メインプロセスの場合はTrue、そうでない場合はFalse
    """

    return not is_distributed() or dist.get_rank() == 0

# 検証
is_main_process()

In [ ]:
def config_to_name(cfg: Config) -> str:
    filename = cfg._filename
    filename = filename.replace("configs/", "")
    filename = filename.replace(".py", "")
    filename = filename.replace("/", "_")
    return filename

In [ ]:
def sync_string(value: str):
    """
    全プロセス間で文字列を同期する

    Args:
        value (str): 同期する文字列
    Returns:
        str: 同期された文字列
    """

    # 分散設定でない場合、そのまま返す
    # True
    if not is_distributed():
        return value

    # 文字列をバイト列にエンコード
    bytes_value = value.encode("utf-8")

    # 最大長256バイトのテンソルを作成
    max_len = 256
    bytes_tensor = torch.zeros(max_len, dtype=torch.uint8).cuda()

    # エンコードしたバイト列を確保したテンソルにコピー
    bytes_tensor[: len(bytes_value)] = torch.tensor(
        list(bytes_value), dtype=torch.uint8
    )

    # ランク0のプロセスから全プロセスにブロードキャスト
    torch.distributed.broadcast(bytes_tensor, 0)

    # バイト列をデコードして文字列に変換
    synced_value = bytes_tensor.cpu().numpy().tobytes().decode("utf-8").rstrip("\x00")

    # 同期された文字列を返す
    return synced_value

In [ ]:
def create_experiment_workspace(
    output_dir: str, model_name: str = None, config: dict = None, exp_name: str = None
) -> tuple[str, str]:
    """
    実験のログを保存するフォルダを作成する
    config.txtやlog.txtなどの実験ごとのログファイルが入る

    Args:
        output_dir (str): 出力ディレクトリのパス
        model_name (str): モデルの名前
        exp_name (str): 実験の名前、Noneの場合はデフォルトを使用
    Returns:
        tuple[str, str]: 実験の名前とディレクトリのパス
    """
    log_.info(f"create_experiment_workspace {output_dir=} {model_name=} {config=} {exp_name=}")

    # 実験の名前が指定されていない場合
    # True
    if exp_name is None:
        # 現在の日時をyyMMdd_HHmmss形式で取得
        experiment_index = datetime.now().strftime("%y%m%d_%H%M%S")

        # すべてのプロセスで同期
        experiment_index = sync_string(experiment_index)

        # モデル名をフォーマット
        # vae_train_video_dc_ae -> -vae_train_video_dc_ae
        model_name = (
            "-" + model_name.replace("/", "-") if model_name is not None else ""
        )

        # 実験の名前を作成
        # NOTE: プロセス間で一意
        exp_name = f"{experiment_index}{model_name}"

    # 実験ディレクトリのパスを作成
    exp_dir = f"{output_dir}/{exp_name}"

    # ランク0のプロセスの場合
    # True
    if is_main_process():

        # 実験ディレクトリを作成
        os.makedirs(exp_dir, exist_ok=True)

        # 訓練の設定をconfig.txtとして保存
        with open(f"{exp_dir}/config.txt", "w", encoding="utf-8") as f:
            # JSON形式書き出し
            json.dump(config, f, indent=4)

    return exp_name, exp_dir


exp_name, exp_dir = create_experiment_workspace(
    cfg.get("outputs", "./outputs"),
    model_name=config_to_name(cfg),
    config=cfg.to_dict(),
)

### 実験ディレクトリの権限を変更

プロセス間でディレクトリを共有・編集可能にする

In [ ]:
def is_pipeline_enabled(plugin_type: str, plugin_config: dict) -> bool:
    """
    パイプライン並列が有効かどうかを判定する
    パイプライン並列は、モデルを層単位で分割し、各層を異なるGPUで処理する手法

    Args:
        plugin_type (str): プラグインのタイプ
        plugin_config (dict): プラグインの設定
    Returns:
        bool: パイプライン並列が有効な場合はTrue、そうでない場合はFalse
    """

    # zero2の場合はデータ並列なのでFalse
    return plugin_type == "hybrid" and plugin_config.get("pp_size", 1) > 1

is_pipeline_enabled(plugin_type, plugin_config)

In [ ]:
def is_log_process(plugin_type: str, plugin_config: dict) -> bool:
    """
    ログを出力するプロセスかどうかを判定する

    Args:
        plugin_type (str): プラグインのタイプ
        plugin_config (dict): プラグインの設定
    Returns:
        bool: ログを出力するプロセスの場合はTrue、そうでない場合はFalse
    """
    # False
    if is_pipeline_enabled(plugin_type, plugin_config):
        # パイプライン並列の最後のランクがログプロセス
        # NOTE: 最終層を担当するプロセスがログを出力するため
        return dist.get_rank() == dist.get_world_size() - 1

    # それ以外はランク0がログプロセス
    return dist.get_rank() == 0

is_log_process(plugin_type, plugin_config)

In [ ]:
# True
if is_log_process(plugin_type, plugin_config):
    log_.info(f"changing {exp_dir} to share")

    # グループ所有権を'share'に変更
    os.system(f"chgrp -R share {exp_dir}")

### ロガーの初期化

In [ ]:
def create_logger(logging_dir: str = None) -> logging.Logger:
    """
    ログファイルとstdoutに書き込むロガーを作成する。メインプロセスのみログを出力する

    Args:
        logging_dir (str): ログファイルを保存するディレクトリ
    Returns:
        logging.Logger: ロガー
    """

    # メインプロセスの場合
    # True
    if is_main_process():

        # ログの追加引数を設定
        additional_args = dict()
        if logging_dir is not None:
            additional_args["handlers"] = [
                # logging.StreamHandler(), # stdoutに出力
                logging.FileHandler(f"{logging_dir}/log.txt"), # ファイルに出力
            ]

        # ログの基本設定
        logging.basicConfig(
            level=logging.INFO,
            # 例: [2025-01-01 12:34:56] ここにメッセージ
            format="[\033[34m%(asctime)s\033[0m] %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
            **additional_args,
        )

        # ロガーを取得
        logger = logging.getLogger(__name__)

        # ログディレクトリが指定されている場合
        if logging_dir is not None:
            # ログを実行
            logger.info("Experiment directory created at %s", logging_dir)

    # それ以外のプロセスの場合
    else:
        # ロガーを取得
        logger = logging.getLogger(__name__)

        # NullHandlerを追加してログを無効化
        logger.addHandler(logging.NullHandler())

    return logger

logger = create_logger(exp_dir)

# 設定をログに出力
logger.info("Training configuration:\n %s", pformat(cfg.to_dict()))

In [ ]:
def log_message(*args, level: str = "info"):
    """
    ロガーにメッセージをログする

    Args:
        *args: ログに記録するメッセージ
        level (str): ログレベル
    """
    logger = logging.getLogger(__name__)
    if level == "info":
        logger.info(*args)
    elif level == "warning":
        logger.warning(*args)
    elif level == "error":
        logger.error(*args)
    elif level == "print":
        print(*args)
    else:
        raise ValueError(f"Invalid logging level: {level}")

### TensorboardとWandBの初期化

In [ ]:
def create_tensorboard_writer(exp_dir: str) -> SummaryWriter:
    """
    テンソルボードのライターを作成する

    Args:
        exp_dir (str): テンソルボードログを保存するディレクトリ
    Returns:
        SummaryWriter: テンソルボードのライター
    """
    # テンソルボードのログディレクトリを作成
    tensorboard_dir = f"{exp_dir}/tensorboard"
    os.makedirs(tensorboard_dir, exist_ok=True)

    # テンソルボードのライターを初期化
    writer = SummaryWriter(tensorboard_dir)
    return writer

In [ ]:
# TensorBoardのライターの変数を初期化
tb_writer = None

# マスタープロセスの場合
# True
if coordinator.is_master():

    # TensorBoardのライターを作成
    tb_writer = create_tensorboard_writer(exp_dir)

    # wandbを使用する場合
    # False
    if cfg.get("wandb", False):

        # wandbを初期化
        wandb.init(
            project=cfg.get("wandb_project", "Open-Sora"),
            name=cfg.get("wandb_expr_name", exp_name),
            config=cfg.to_dict(),
            dir=exp_dir,
        )

### テキストデータセットクラスの作成

In [ ]:
# データセットのディレクトリを上書き
cfg.dataset["data_path"] = PEXELS_DATASET_DIR
cfg.dataset

In [ ]:
def read_file(input_path, memory_efficient=False):
    """
    ファイルを読み込む
    TextDatasetクラスの初期化時に使用

    Args:
        input_path (str): 入力ファイルのパス
        memory_efficient (bool): メモリ効率モードで読み込むかどうか
    Returns:
        pd.DataFrame: 読み込んだデータフレーム
    """

    log_.info(f"read_file {input_path=} {memory_efficient=}")

    # True
    if input_path.endswith(".csv"):
        assert not memory_efficient, "Memory efficient mode is not supported for CSV files"

        # PandasでCSVファイルを読み込む
        # /workspaces/open-sora/Open-Sora/pexels_5.csv
        return pd.read_csv(input_path)

    # False
    elif input_path.endswith(".parquet"):
        columns = None
        if memory_efficient:
            columns = ["path", "num_frames", "height", "width", "aspect_ratio", "fps", "resolution"]
        if SUPPORT_DASK:
            ret = dd.read_parquet(input_path, columns=columns).compute()
        else:
            ret = pd.read_parquet(input_path, columns=columns)
        return ret

    # False
    else:
        raise NotImplementedError(f"Unsupported file format: {input_path}")

In [ ]:
VALID_KEYS = ("neg", "path")

# @DATASETS.register_module("text")
class TextDataset(torch.utils.data.Dataset):
    """
    テキストデータ用のPyTorchのデータセットクラス
    VideoTextDatasetで継承される
    """

    def __init__(self, data_path: str = None, tokenize_fn: callable = None, fps_max: int = 16, vmaf: bool = False, memory_efficient: bool = False, **kwargs):
        log_.info(f"TextDataset.__init__ {data_path=} {tokenize_fn=} {fps_max=} {vmaf=} {memory_efficient=} {kwargs=}")

        # データパスを読み込み
        self.data_path = data_path
        self.data = read_file(data_path, memory_efficient=memory_efficient)

        # メモリ効率モードを設定
        self.memory_efficient = memory_efficient

        # トークナイズ関数を設定
        self.tokenize_fn = tokenize_fn

        self.vmaf = vmaf

        # 最大FPSが指定されている場合
        if fps_max is not None:
            # 最大FPSを設定
            self.fps_max = fps_max
        else:
            self.fps_max = 999999999

    def to_efficient(self):
        if self.memory_efficient:
            addition_data_path = self.data_path.split(".")[0]
            self._data = self.data
            self.data = EfficientParquet(self._data, addition_data_path)

    def getitem(self, index: int) -> dict:
        log_.info(f"TextDataset.getitem {index=}")
        ret = dict()
        sample = self.data.iloc[index].to_dict()
        sample_fps = sample.get("fps", np.nan)
        new_fps, sampling_interval = map_target_fps(sample_fps, self.fps_max)
        ret.update({"sampling_interval": sampling_interval})

        if "text" in sample:
            ret["text"] = sample.pop("text")
            postfixs = []
            if new_fps != 0 and self.fps_max < 999:
                postfixs.append(f"{new_fps} FPS")
            if self.vmaf and "score_vmafmotion" in sample and not np.isnan(sample["score_vmafmotion"]):
                postfixs.append(f"{int(sample['score_vmafmotion'] + 0.5)} motion score")
            postfix = " " + ", ".join(postfixs) + "." if postfixs else ""
            ret["text"] = ret["text"] + postfix
            if self.tokenize_fn is not None:
                ret.update({k: v.squeeze(0) for k, v in self.tokenize_fn(ret["text"]).items()})

        if "ref" in sample:  # i2v & v2v reference
            ret["ref"] = sample.pop("ref")

        # name of the generated sample
        if "name" in sample:  # sample name (`dataset_idx`)
            ret["name"] = sample.pop("name")
        else:
            ret["index"] = index  # use index for name
        valid_sample = {k: v for k, v in sample.items() if k in VALID_KEYS}
        ret.update(valid_sample)
        return ret

    def __getitem__(self, index):
        return self.getitem(index)

    def __len__(self):
        return len(self.data)

### 動画を読み込む関数を作成

In [ ]:
def _read_from_stream(video_frames, container: "av.container.Container", start_offset: float, end_offset: float, pts_unit: str, stream: "av.stream.Stream", stream_name: dict[str, int | tuple[int, ...] | list[int] | None], filename: str | None = None) -> list["av.frame.Frame"]:
    """
    指定されたストリームからフレームのリストを読み込む
    read_video_av関数で使用

    Args:
        video_frames: フレームを格納するリスト
        container (av.container.Container): メディアコンテナ
        start_offset (float): 読み込み開始オフセット
        end_offset (float): 読み込み終了オフセット
        pts_unit (str): PTSの単位 ("sec"または"pts")
        stream (av.stream.Stream): 読み込むストリーム
        stream_name (dict): ストリーム名の辞書
        filename (str | None): ファイル名（デバッグ用）
    Returns:
        list[av.frame.Frame]: 読み込んだフレームのリスト
    """
    log_.info(f"_read_from_stream {start_offset=} {end_offset=} {pts_unit=} {stream=} {stream_name=} {filename=}")

    if pts_unit == "sec":
        # TODO: we should change all of this from ground up to simply take
        # sec and convert to MS in C++
        start_offset = int(math.floor(start_offset * (1 / stream.time_base)))
        if end_offset != float("inf"):
            end_offset = int(math.ceil(end_offset * (1 / stream.time_base)))
    else:
        warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")

    should_buffer = True
    max_buffer_size = 5
    if stream.type == "video":
        # DivX-style packed B-frames can have out-of-order pts (2 frames in a single pkt)
        # so need to buffer some extra frames to sort everything
        # properly
        extradata = stream.codec_context.extradata
        # overly complicated way of finding if `divx_packed` is set, following
        # https://github.com/FFmpeg/FFmpeg/commit/d5a21172283572af587b3d939eba0091484d3263
        if extradata and b"DivX" in extradata:
            # can't use regex directly because of some weird characters sometimes...
            pos = extradata.find(b"DivX")
            d = extradata[pos:]
            o = re.search(rb"DivX(\d+)Build(\d+)(\w)", d)
            if o is None:
                o = re.search(rb"DivX(\d+)b(\d+)(\w)", d)
            if o is not None:
                should_buffer = o.group(3) == b"p"
    seek_offset = start_offset
    # some files don't seek to the right location, so better be safe here
    seek_offset = max(seek_offset - 1, 0)
    if should_buffer:
        # FIXME this is kind of a hack, but we will jump to the previous keyframe
        # so this will be safe
        seek_offset = max(seek_offset - max_buffer_size, 0)
    try:
        # TODO check if stream needs to always be the video stream here or not
        container.seek(seek_offset, any_frame=False, backward=True, stream=stream)
    except av.AVError as e:
        print(f"[Warning] Error while seeking video {filename}: {e}")
        return []

    # == main ==
    buffer_count = 0
    frames_pts = []
    cnt = 0
    try:
        for _idx, frame in enumerate(container.decode(**stream_name)):
            frames_pts.append(frame.pts)
            video_frames[cnt] = frame.to_rgb().to_ndarray()
            cnt += 1
            if cnt >= len(video_frames):
                break
            if frame.pts >= end_offset:
                if should_buffer and buffer_count < max_buffer_size:
                    buffer_count += 1
                    continue
                break
    except av.AVError as e:
        print(f"[Warning] Error while reading video {filename}: {e}")

    # garbage collection for thread leakage
    container.close()
    del container
    # NOTE: manually garbage collect to close pyav threads
    gc.collect()

    # ensure that the results are sorted wrt the pts
    # NOTE: here we assert frames_pts is sorted
    start_ptr = 0
    end_ptr = cnt
    while start_ptr < end_ptr and frames_pts[start_ptr] < start_offset:
        start_ptr += 1
    while start_ptr < end_ptr and frames_pts[end_ptr - 1] > end_offset:
        end_ptr -= 1
    if start_offset > 0 and start_offset not in frames_pts[start_ptr:end_ptr]:
        # if there is no frame that exactly matches the pts of start_offset
        # add the last frame smaller than start_offset, to guarantee that
        # we will have all the necessary data. This is most useful for audio
        if start_ptr > 0:
            start_ptr -= 1
    result = video_frames[start_ptr:end_ptr].copy()
    return result

In [ ]:
def read_video_av(filename: str, start_pts: float | Fraction = 0, end_pts: float | Fraction | None = None, pts_unit: str = "pts", output_format: str = "THWC") -> tuple[torch.Tensor, torch.Tensor, dict]:
    """
    ファイルから動画を読み込み、動画フレームと音声フレームの両方を返す

    torchvision.io.video.read_videoから修正された実装:
    1. 音声フレームを抽出せず、aframesは空で返す
    2. チェックを削除し、pyavのみをサポート
    3. container.close()とgc.collect()を追加してスレッドリークを回避
    4. メモリリークをできるだけ回避

    Args:
        filename (str): 動画ファイルへのパス
        start_pts (int if pts_unit = 'pts', float / Fraction if pts_unit = 'sec', optional):
            動画の開始プレゼンテーション時間
        end_pts (int if pts_unit = 'pts', float / Fraction if pts_unit = 'sec', optional):
            終了プレゼンテーション時間
        pts_unit (str, optional): start_ptsとend_ptsの値が解釈される単位、'pts'または'sec'。デフォルトは'pts'。
        output_format (str, optional): 出力動画テンソルの形式。"THWC"（デフォルト）または"TCHW"のいずれか。

    Returns:
        vframes (Tensor[T, H, W, C] or Tensor[T, C, H, W]): `T`動画フレーム
        aframes (Tensor[K, L]): 音声フレーム、`K`はチャネル数、`L`はポイント数
        info (dict): 動画と音声のメタデータ。video_fps（float）とaudio_fps（int）のフィールドを含む場合がある
    """
    log_.info(f"read_video_av {filename=} {start_pts=} {end_pts=} {pts_unit=} {output_format=}")

    # 1) 出力形式のチェック

    output_format = output_format.upper()

    # THWCかTCHWのみサポート
    # T = time, H = height, W = width, C = channels
    if output_format not in ("THWC", "TCHW"):
        raise ValueError(f"output_format should be either 'THWC' or 'TCHW', got {output_format}.")

    # 2) ファイルの存在チェック

    if not os.path.exists(filename):
        raise RuntimeError(f"File not found: {filename}")

    # 3) pyavが使用可能かチェック

    # torchvisionのメソッド
    assert get_video_backend() == "pyav", "pyav backend is required for read_video_av"

    # torchvision.datasets.videoのメソッド
    _check_av_available()

    # 4) 開始PTSと終了PTSを検証

    # pts = Presentation Time Stamp
    # 各フレームの再生時刻を示すタイムスタンプ

    # 終了PTSが指定されていない場合、無限大に設定
    if end_pts is None:
        end_pts = float("inf")

    # 終了PTSが開始PTSより小さい場合、エラーをスロー
    if end_pts < start_pts:
        raise ValueError(f"end_pts should be larger than start_pts, got start_pts={start_pts} and end_pts={end_pts}")

    # 5) 動画情報の取得

    info = {}

    # PyAVでコンテナを開く
    # コンテナは、動画や音声などのメディアデータを格納するファイル形式
    container = av.open(filename, metadata_errors="ignore")

    # コンテナから平均フレームレートを取得
    video_fps = container.streams.video[0].average_rate

    # floatに変換して動画情報に追加
    if video_fps is not None:
        info["video_fps"] = float(video_fps)

    # 6) 動画フレームの読み込み

    # 0番目の動画ストリームをデコードするイテレータを作成
    iter_video = container.decode(**{"video": 0})

    # 最初のフレームを取得してRGB形式に変換し、NumPy配列に変換
    frame = next(iter_video).to_rgb().to_ndarray()

    # 形状は(H, W, C)なので、高さと幅を取得
    height, width = frame.shape[:2]

    # 動画ストリームの総フレーム数を取得
    total_frames = container.streams.video[0].frames

    # 総フレーム数が0の場合、最大フレーム数を使用
    if total_frames == 0:
        total_frames = MAX_NUM_FRAMES
        warnings.warn(f"total_frames is 0, using {MAX_NUM_FRAMES} as a fallback")

    # コンテナを閉じてメモリリークを回避
    container.close()
    del container

    # 動画フレームを格納するNumPy配列を初期化
    # NOTE: np.zerosは実際にはメモリを割り当てないためメモリリークを回避できる
    video_frames = np.zeros((total_frames, height, width, 3), dtype=np.uint8)

    try:
        # PyAVでコンテナを再度開く
        container = av.open(filename, metadata_errors="ignore")

        assert container.streams.video is not None

        # 動画フレームをストリームから読み込む
        video_frames = _read_from_stream(
            video_frames,
            container,
            start_pts,
            end_pts,
            pts_unit,
            container.streams.video[0],
            {"video": 0},
            filename=filename,
        )
    except av.AVError as e:
        print(f"[Warning] Error while reading video {filename}: {e}")

    # 7) 出力テンソルの作成

    # NumPy配列からPyTorchテンソルを作成
    vframes = torch.from_numpy(video_frames).clone()
    del video_frames

    # 出力形式がTCHWの場合、テンソルの次元を並べ替え
    if output_format == "TCHW":
        # [T,H,W,C] --> [T,C,H,W]
        vframes = vframes.permute(0, 3, 1, 2)

    # 8) 空の音声フレームテンソルを作成
    aframes = torch.empty((1, 0), dtype=torch.float32)

    return vframes, aframes, info

In [ ]:
def read_video(video_path, backend="av"):
    """
    動画ファイルを読み込み、動画フレームと動画情報を返す

    Args:
        video_path (str): 動画ファイルへのパス
        backend (str): 使用するバックエンド、"cv2"または"av"。デフォルトは"av"。
    Returns:
        vframes (Tensor): 動画フレームのテンソル
        vinfo (dict): 動画のメタデータ
    """
    log_.info(f"read_video {video_path=} {backend=}")

    # False
    if backend == "cv2":
        vframes, vinfo = read_video_cv2(video_path)

    # True
    elif backend == "av":
        vframes, _, vinfo = read_video_av(filename=video_path, pts_unit="sec", output_format="TCHW")

    # False
    else:
        raise ValueError

    return vframes, vinfo

### 動画を時系列でクロップする関数を作成

In [ ]:
class TemporalRandomCrop(object):
    """Temporally crop the given frame indices at a random location.

    Args:
            size (int): Desired length of frames will be seen in the model.
    """

    def __init__(self, size):
        self.size = size

    def __call__(self, total_frames):
        rand_end = max(0, total_frames - self.size - 1)
        begin_index = random.randint(0, rand_end)
        end_index = min(begin_index + self.size, total_frames)
        return begin_index, end_index

In [ ]:
def temporal_random_crop(
    vframes: torch.Tensor, num_frames: int, frame_interval: int, return_frame_indices: bool = False
) -> torch.Tensor | tuple[torch.Tensor, np.ndarray]:
    # temporal_sample = video_transforms.temporalrandomcrop(num_frames * frame_interval)
    temporal_sample = TemporalRandomCrop(num_frames * frame_interval)

    total_frames = len(vframes)
    start_frame_ind, end_frame_ind = temporal_sample(total_frames)

    assert (
        end_frame_ind - start_frame_ind >= num_frames
    ), f"Not enough frames to sample, {end_frame_ind} - {start_frame_ind} < {num_frames}"

    frame_indices = np.linspace(start_frame_ind, end_frame_ind - 1, num_frames, dtype=int)
    video = vframes[frame_indices]
    if return_frame_indices:
        return video, frame_indices
    else:
        return video

### 動画をテンソルに変換する関数を作成

In [ ]:
def _is_tensor_video_clip(clip):
    """
    動画クリップが4Dテンソルかどうかを判定する
    (T, H, W, C)または(T, C, H, W)の形式

    Args:
        clip: 動画クリップ
    Returns:
        bool: 4Dテンソルの場合はTrue、そうでない場合はFalse
    """

    # テンソルでない場合、エラーをスロー
    if not torch.is_tensor(clip):
        raise TypeError("clip should be Tensor. Got %s" % type(clip))

    # 4次元テンソルでない場合、エラーをスロー
    if not clip.ndimension() == 4:
        raise ValueError("clip should be 4D. Got %dD" % clip.dim())

    return True

In [ ]:
def to_tensor(clip):
    """
    uint8テンソル（0〜255の範囲）をfloatテンソル（0.0〜1.0の範囲）に変換する

    Args:
        clip (torch.tensor, dtype=torch.uint8): サイズは(T, C, H, W)
    Returns:
        clip (torch.tensor, dtype=torch.float): サイズは(T, C, H, W)
    """

    # 動画クリップが4Dテンソルかどうかを判定
    _is_tensor_video_clip(clip)

    # データ型がuint8でない場合、エラーをスロー
    if not clip.dtype == torch.uint8:
        raise TypeError("clip tensor should have data type uint8. Got %s" % str(clip.dtype))

    # 255.0で割る
    # これにより、値が0.0から1.0の範囲に正規化される
    return clip.float() / 255.0

In [ ]:
class ToTensorVideo:
    """
    uint8テンソル（0〜255の範囲）をfloatテンソル（0.0〜1.0の範囲）に変換するモジュール
    to_tensor関数のラッパー
    """

    def __init__(self):
        pass

    def __call__(self, clip):
        """
        Args:
            clip (torch.tensor, dtype=torch.uint8): サイズが(T, C, H, W)
        Return:
            clip (torch.tensor, dtype=torch.float): サイズが(T, C, H, W)
        """
        return to_tensor(clip)

    def __repr__(self) -> str:
        return self.__class__.__name__

### リサイズとクロップするモジュールを作成

In [ ]:
def resize(clip, target_size, interpolation_mode):
    """
    動画クリップを指定されたサイズにリサイズする

    Args:
        clip (torch.tensor): サイズが(T, C, H, W)
        target_size (tuple): 目標のサイズ (height, width)
        interpolation_mode (str): 補間モード、'nearest'、'bilinear'、'bicubic'など
    Returns:
        torch.tensor: リサイズされた動画クリップ、サイズは(T, C, target_height, target_width)
    """
    log_.info(f"resize {target_size=} {interpolation_mode=}")

    if len(target_size) != 2:
        raise ValueError(f"target size should be tuple (height, width), instead got {target_size}")

    # 動画クリップをリサイズ
    return torch.nn.functional.interpolate(clip, size=target_size, mode=interpolation_mode, align_corners=False)


In [ ]:
def crop(clip, i, j, h, w):
    """
    動画クリップを指定された位置でクロップする

    Args:
        clip (torch.tensor): 動画クリップ、サイズは (T, C, H, W)
        i (int): クロップの左上隅の高さの位置
        j (int): クロップの左上隅の幅の位置
        h (int): クロップの高さ
        w (int): クロップの幅
    Returns:
        torch.tensor: クロップされた動画クリップ、サイズは (T, C, h, w)
    """
    log_.info(f"crop {i=} {j=} {h=} {w=}")

    # 4Dテンソルでない場合、エラーをスロー
    if len(clip.size()) != 4:
        raise ValueError("clip should be a 4D tensor")

    # HとWの次元に対してクロップを適用
    return clip[..., i : i + h, j : j + w]

In [ ]:
def resize_crop_to_fill(clip, target_size):
    """
    動画クリップを指定されたサイズにリサイズし、中央でクロップする
    resize関数とcrop関数のラッパー

    Args:
        clip (torch.tensor): 動画クリップ、サイズは (T, C, H, W)
        target_size (tuple): 目標のサイズ (height, width)
    Returns:
        torch.tensor: リサイズおよびクロップされた動画クリップ、サイズは (T, C, target_height, target_width)
    """
    log_.info(f"resize_crop_to_fill {target_size=}")

    # 4Dテンソルでない場合、エラーをスロー
    if not _is_tensor_video_clip(clip):
        raise ValueError("clip should be a 4D torch.tensor")

    # 元の高さと幅を取得
    h, w = clip.size(-2), clip.size(-1)

    # 目標の高さと幅を取得
    th, tw = target_size[0], target_size[1]

    # リサイズ後の比率を計算
    rh, rw = th / h, tw / w

    # 高さの比率が幅の比率より大きい場合
    if rh > rw:
        # 目標の高さは維持し、幅を調整
        sh, sw = th, round(w * rh)

        # 動画クリップをリサイズ
        clip = resize(clip, (sh, sw), "bilinear")

        # クロップの左上の位置を計算
        i = 0
        j = int(round(sw - tw) / 2.0)

    # 幅の比率が高さの比率より大きい場合
    else:
        # 目標の幅は維持し、高さを調整
        sh, sw = round(h * rw), tw

        # 動画クリップをリサイズ
        clip = resize(clip, (sh, sw), "bilinear")

        # クロップの左上の位置を計算
        i = int(round(sh - th) / 2.0)
        j = 0

    assert i + th <= clip.size(-2) and j + tw <= clip.size(-1)

    # クロップを適用して返す
    return crop(clip, i, j, th, tw)

In [ ]:
class ResizeCrop:
    """
    動画クリップを指定されたサイズにリサイズし、クロップするモジュール
    resize_crop_to_fill関数のラッパー
    """

    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, clip):
        clip = resize_crop_to_fill(clip, self.size)
        return clip

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(size={self.size})"


In [ ]:
def get_transforms_video(name="center", image_size=(256, 256)):
    """
    動画の前処理変換用のモジュールを取得する

    Args:
        name (str): 変換方法
            - center: 中央クロップ
            - resize_crop: リサイズしてクロップ
            - rand_size_crop: ランダムサイズクロップ
        image_size (tuple): 変換後の画像サイズ (height, width)
    Returns:
        transform_video (torchvision.transforms.Compose): 動画の前処理変換モジュール
    """
    log_.info(f"get_transforms_video {name=} {image_size=}")

    # False
    if name is None:
        return None

    # False
    elif name == "center":
        assert image_size[0] == image_size[1], "image_size must be square for center crop"
        transform_video = transforms.Compose(
            [
                ToTensorVideo(),  # TCHW
                UCFCenterCropVideo(image_size[0]),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
            ]
        )

    # True
    elif name == "resize_crop":

        # Composeで変換モジュールを連結
        transform_video = transforms.Compose(
            [
                ToTensorVideo(),  # 0~255の値を0~1の範囲に正規化
                ResizeCrop(image_size), # リサイズしてクロップ
                transforms.Normalize(
                    mean=[0.5, 0.5, 0.5],
                    std=[0.5, 0.5, 0.5],
                    inplace=True
                ), # 平均0.5、標準偏差0.5で正規化
            ]
        )

    # False
    elif name == "rand_size_crop":
        transform_video = transforms.Compose(
            [
                ToTensorVideo(),  # TCHW
                video_transforms.RandomSizedCrop(image_size),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
            ]
        )
    else:
        raise NotImplementedError(f"Transform {name} not implemented")
    return transform_video

### 動画とテキストのデータセットクラスを作成

In [ ]:
def is_img(path):
    """
    ファイルが画像ファイルかどうかを判定する

    Args:
        path (str): ファイルのパス
    Returns:
        bool: 画像ファイルの場合はTrue、そうでない場合はFalse
    """
    # 拡張子を取得
    ext = os.path.splitext(path)[-1].lower()

    # torchvision.datasets.forlder.IMG_EXTENSIONSで検証
    return ext in IMG_EXTENSIONS

In [ ]:
# @DATASETS.register_module("video_text")
class VideoTextDataset(TextDataset):
    def __init__(
        self,
        transform_name: str = None,
        bucket_class: str = "Bucket",
        rand_sample_interval: int = None,  # random sample_interval value from [1, min(rand_sample_interval, video_allowed_max)]
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.transform_name = transform_name
        self.bucket_class = bucket_class
        self.rand_sample_interval = rand_sample_interval

    def get_image(self, index: int, height: int, width: int) -> dict:
        sample = self.data.iloc[index]
        path = sample["path"]
        # loading
        image = pil_loader(path)

        # transform
        transform = get_transforms_image(self.transform_name, (height, width))
        image = transform(image)

        # CHW -> CTHW
        video = image.unsqueeze(1)

        return {"video": video}

    def get_video(self, index: int, num_frames: int, height: int, width: int, sampling_interval: int) -> dict:
        """
        動画を取得する

        Args:
            index (int): データセットのインデックス
            num_frames (int): 取得するフレーム数
            height (int): 動画の高さ
            width (int): 動画の幅
            sampling_interval (int): フレーム間のサンプリング間隔
        Returns:
            dict: 動画のデータを含む辞書
        """

        # データフレームからインデックスに対応するサンプルを取得
        sample = self.data.iloc[index]

        # pathカラムからファイルパスを取得
        path = sample["path"]

        # 動画を読み込む
        vframes, vinfo = read_video(path, backend="av")

        # ランダムサンプリング間隔が指定されている場合
        if self.rand_sample_interval is not None:
            # 最大サンプリング間隔を計算
            video_allowed_max = min(len(vframes) // num_frames, self.rand_sample_interval)
            # 1から最大サンプリング間隔までのランダムなサンプリング間隔を選択（int型）
            sampling_interval = random.randint(1, video_allowed_max)

        # サンプリング間隔に基づいて動画を時間的にランダムクロップ
        video = temporal_random_crop(vframes, num_frames, sampling_interval)

        # vframesをクローンしてメモリリークを回避
        video = video.clone()
        del vframes

        # 動画変換のモジュールを取得
        transform = get_transforms_video(self.transform_name, (height, width))

        # 動画に変換を適用
        # 0~1の範囲に変換され、リサイズおよびクロップされ、平均と標準偏差で正規化される
        video = transform(video)

        # 次元を並べ替え
        # TCHW -> CTHW
        video = video.permute(1, 0, 2, 3)

        # 出力を辞書に格納
        ret = {"video": video}

        return ret

    def get_image_or_video(self, index: int, num_frames: int, height: int, width: int, sampling_interval: int) -> dict:
        """
        画像または動画を取得する
        get_videoのラッパー関数

        Args:
            index (int): データセットのインデックス
            num_frames (int): 取得するフレーム数（動画の場合）
            height (int): 画像/動画の高さ
            width (int): 画像/動画の幅
            sampling_interval (int): フレーム間のサンプリング間隔（動画の場合）
        Returns:
            dict: 画像または動画のデータを含む辞書
        """

        # データフレームからインデックスに対応するサンプルを取得
        sample = self.data.iloc[index]

        # pathカラムからファイルパスを取得
        path = sample["path"]

        # 画像ファイルである場合
        if is_img(path):

            # 画像を取得して返す
            return self.get_image(index, height, width)

        # 動画ファイルである場合、動画を取得して返す
        return self.get_video(index, num_frames, height, width, sampling_interval)

    def getitem(self, index: str) -> dict:
        # a hack to pass in the (time, height, width) info from sampler
        index, num_frames, height, width = [int(val) for val in index.split("-")]
        ret = dict()
        ret.update(super().getitem(index))
        try:
            ret.update(self.get_image_or_video(index, num_frames, height, width, ret["sampling_interval"]))
        except Exception as e:
            path = self.data.iloc[index]["path"]
            print(f"video {path}: {e}")
            return None
        return ret

    def __getitem__(self, index):
        return self.getitem(index)


# データセットを構築
dataset = VideoTextDataset(
    type = "video_text",
    transform_name = "resize_crop",
    data_path = PEXELS_CSV_PATH,
    fps_max = 24,
)

logger.info("Dataset contains %s samples.", len(dataset))

# 検証
sample = dataset.get_image_or_video(0, num_frames=16, height=256, width=256, sampling_interval=2)

# (3, 16, 256, 256)
sample["video"].shape

In [ ]:
# 8枚の動画フレームをグリッド形式で表示
grid = torchvision.utils.make_grid(sample["video"][:, ::2, :, :].permute(1, 0, 2, 3), nrow=8, padding=2)
plt.figure(figsize=(12, 6))
plt.axis("off")
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.show()

### バケツクラスを作成

In [ ]:
# 動画のアスペクト比リスト
# width:height
ASPECT_RATIO_LD_LIST = [
    "2.39:1",  # cinemascope, 2.39
    "2:1",  # rare, 2
    "16:9",  # rare, 1.89
    "1.85:1",  # american widescreen, 1.85
    "9:16",  # popular, 1.78
    "5:8",  # rare, 1.6
    "3:2",  # rare, 1.5
    "4:3",  # classic, 1.33
    "1:1",  # square
]

In [ ]:
((256 * 256 * (16 / 9))**(1/2) // (16 * 16)) * 16

In [ ]:
def get_aspect_ratios_dict(total_pixels: int = 256 * 256, training: bool = True) -> dict[str, tuple[int, int]]:
    """
    指定された総ピクセル数に基づいて、事前定義されたアスペクト比の辞書を取得する

    Args:
        total_pixels (int): 目標の総ピクセル数（デフォルトは256 * 256）
        training (bool): トレーニングモードかどうか（デフォルトはTrue）
    Returns:
        dict[str, tuple[int, int]]:
            アスペクト比の辞書、キーはアスペクト比の文字列、値は(高さ, 幅)のタプル
    """
    log_.info(f"get_aspect_ratios_dict {total_pixels=} {training=}")

    # オートエンコーダーの空間圧縮率を取得
    # 16
    D = int(os.environ.get("AE_SPATIAL_COMPRESSION", 16))

    # 出力用の辞書
    aspect_ratios_dict = {}
    aspect_ratios_vertical_dict = {}

    # アスペクト比のリストでループ
    for ratio in ASPECT_RATIO_LD_LIST:

        # アスペクト比の文字列を分割して数値に変換
        # 例: "16:9" -> 16.0, 9.0
        width_ratio, height_ratio = map(float, ratio.split(":"))

        # アスペクト比を維持しつつ、面積がtotal_pixelsに近い幅と高さを計算
        # 16:9の場合、width=336px, height=192px
        width = int(math.sqrt(total_pixels * (width_ratio / height_ratio)) // D) * D
        height = int((total_pixels / width) // D) * D

        # 訓練モードの場合、より誤差の少ない解像度を採用する
        if training:
            # 目標の総ピクセル数との誤差
            diff = abs(height * width - total_pixels)

            # 上下左右に16ピクセルずつ調整した候補を生成
            candidate = [
                (height - D, width),
                (height + D, width),
                (height, width - D),
                (height, width + D),
            ]

            # 最も誤差の少ない候補を探索して採用
            for h, w in candidate:
                if abs(h * w - total_pixels) < diff:
                    height, width = h, w
                    diff = abs(h * w - total_pixels)

        # 重複しない場合、辞書に追加
        if (height, width) not in aspect_ratios_dict.values() or not training:
            # 辞書にアスペクト比と対応する(高さ, 幅)を追加
            aspect_ratios_dict[ratio] = (height, width)

            # 縦向きのアスペクト比も計算して辞書に追加
            vertial_ratios = ":".join(ratio.split(":")[::-1])
            aspect_ratios_vertical_dict[vertial_ratios] = (width, height)

    # 縦向きのアスペクト比をメインの辞書に統合
    aspect_ratios_dict.update(aspect_ratios_vertical_dict)

    return aspect_ratios_dict

# 検証
get_aspect_ratios_dict()

In [ ]:
def get_num_pexels_from_name(resolution: str) -> int:
    """
    Pexelsの動画解像度の名前から総ピクセル数を取得する

    Args:
        resolution (str): 解像度の名前（例: "256px_ar1:1")
    Returns:
        int: 総ピクセル数
    """

    # 解像度の名前からピクセル数の部分を抽出
    resolution = resolution.split("_")[0]

    # pxで終わる場合
    # True
    if resolution.endswith("px"):

        # ピクセル数を整数に変換
        size = int(resolution[:-2])

        # 総ピクセル数を計算
        num_pexels = size * size

    # pで終わる場合
    # False
    elif resolution.endswith("p"):
        size = int(resolution[:-1])
        num_pexels = int(size * size / 9 * 16)
    else:
        raise ValueError(f"Invalid resolution {resolution}")
    return num_pexels

# 検証
get_num_pexels_from_name("256px")  # 65536

In [ ]:
def get_resolution_with_aspect_ratio(
    resolution: str,
) -> tuple[int, dict[str, tuple[int, int]]]:
    """
    アスペクト比を考慮した解像度を取得する
    bucket_configから解像度を抽出するために使用される

    Args:
        resolution (str):
            解像度の名前
            形式は"name"または"{name}_{setting}"
            nameは"256px"または"360p"をサポート
            settingは"ar1:1"または"max"をサポート
    Returns:
        tuple[int, dict[str, tuple[int, int]]]: アスペクト比を考慮した解像度
    """
    log_.info(f"get_resolution_with_aspect_ratio {resolution=}")

    # 1) 解像度と設定を分割

    # 256px_ar1:1 -> ["256px", "ar1:1"]
    keys = resolution.split("_")

    if len(keys) == 1:
        resolution = keys[0]
        setting = ""
    else:
        resolution, setting = keys
        assert setting == "max" or setting.startswith(
            "ar"
        ), f"Invalid setting {setting}"

    # 2) 総ピクセル数を取得

    # 256px -> 65536
    num_pexels = get_num_pexels_from_name(resolution)

    # 3) 総ピクセル数に対応するアスペクト比の辞書を取得

    # 65536 -> {"16:9": (192, 336), ...}
    aspect_ratio_dict = get_aspect_ratios_dict(num_pexels)

    # 4) 設定に基づいてアスペクト比をフィルタリング

    # False
    if setting == "max":
        aspect_ratio = max(
            aspect_ratio_dict,
            key=lambda x: aspect_ratio_dict[x][0] * aspect_ratio_dict[x][1],
        )
        aspect_ratio_dict = {aspect_ratio: aspect_ratio_dict[aspect_ratio]}

    # ar1:1
    # True
    elif setting.startswith("ar"):

        # アスペクト比を抽出
        # 1:1
        aspect_ratio = setting[2:]
        assert (
            aspect_ratio in aspect_ratio_dict
        ), f"Aspect ratio {aspect_ratio} not found"

        # アスペクト比の辞書をフィルタリング
        aspect_ratio_dict = {aspect_ratio: aspect_ratio_dict[aspect_ratio]}

    return num_pexels, aspect_ratio_dict

get_resolution_with_aspect_ratio("256px_ar1:1")  # (65536, {'16:9': (192, 336)})

In [ ]:
def map_target_fps(
    fps: float,
    max_fps: float,
) -> tuple[float, int]:
    """
    fpsが高い場合、最大fps未満にマッピングする

    Args:
        fps (float): 元のfps
        max_fps (float): 最大fps
    Returns:
        tuple[float, int]: 新しいfpsとサンプリング間隔
    """
    log_.info(f"map_target_fps {fps=} {max_fps=}")

    # fpsがNaNの場合、0fps・サンプリング間隔1を返す
    if math.isnan(fps):
        return 0, 1

    # fpsがmax_fps未満の場合、そのまま返す
    if fps < max_fps:
        return fps, 1

    # fpsがmax_fps以上の場合、サンプリング間隔を計算
    sampling_interval = math.ceil(fps / max_fps)
    new_fps = math.floor(fps / sampling_interval)

    return new_fps, sampling_interval

# 検証
# 60fpsを24fps未満にマッピングする場合、20fps・サンプリング間隔3が得られる
# (20.0, 3)
map_target_fps(60.0, 24.0)

In [ ]:
def get_ratio(name: str) -> float:
    """
    アスペクト比の名前から比率を取得する

    Args:
        name (str): アスペクト比の名前（例: "16:9"）
    Returns:
        float: アスペクト比の比率（高さ/幅）
    """

    # 文字列を分割して数値に変換
    width, height = map(float, name.split(":"))

    # 比率を計算して返す
    return height / width

# 検証
get_ratio("16:9")  # 0.5625

In [ ]:
def get_closest_ratio(height: float, width: float, ratios: dict) -> str:
    """
    指定された高さと幅に最も近いアスペクト比の名前を取得する
    Args:
        height (float): 高さ
        width (float): 幅
        ratios (dict): アスペクト比の辞書、キーはアスペクト比の名前、値は(高さ, 幅)のタプル
    Returns:
        str: 最も近いアスペクト比の名前
    """

    # 高さと幅からアスペクト比を計算
    aspect_ratio = height / width

    # アスペクト比の辞書から最も近いアスペクト比の名前を取得
    closest_ratio = min(
        ratios.keys(), key=lambda ratio: abs(aspect_ratio - get_ratio(ratio))
    )

    return closest_ratio

# 検証
# 192:336に最も近いアスペクト比は16:9
get_closest_ratio(192, 336, get_aspect_ratios_dict())

In [ ]:
cfg.bucket_config.keys()

In [ ]:
class Bucket:
    """
    動画をバケットに分類するクラス
    訓練の最初に動画をバケツに分類し、ステップごとにバケツを切り替えるために使用する
    オートエンコーダーの場合は単一のバケツ
    MMDiTの場合は複数のバケツがあり、解像度ごとに最適なバッチサイズが決められている
    """

    def __init__(self, bucket_config: dict[str, dict[int, tuple[float, int] | tuple[tuple[float, float], int]]]):
        """
        Args:
            bucket_config (dict): バケット構成を含む辞書

                辞書の形式は以下の通り:
                {
                    "bucket_name": {
                        "time": (probability, batch_size),
                        "time": (probability, batch_size),
                        ...
                    },
                    ...
                }

                または以下の形式:

                {
                    "bucket_name": {
                        "time": ((probability, next_probability), batch_size),
                        "time": ((probability, next_probability), batch_size),
                        ...
                    },
                    ...
                }
                bucket_nameはバケットの名前、timeは動画のフレーム数を示す。
                probabilityは0から1の間の浮動小数点数、batch_sizeは整数である必要がある。
                probabilityがタプルの場合、2番目の値は次のtimeにスキップする確率を示す。
        """
        log_.info(f"Bucket.__init__ {bucket_config=}")

        # bucket_configからアスペクト比を取得
        # "256px_ar1:1" -> {"256px_ar1:1": (65536, {"16:9": (192, 336)}), ...}
        aspect_ratios = {key: get_resolution_with_aspect_ratio(key) for key in bucket_config.keys()}

        # バケットの確率とバッチサイズを格納するOrderedDictで初期化
        bucket_probs = OrderedDict()
        bucket_bs = OrderedDict()

        # アスペクト比に基づいてバケット名をソート（解像度の高い順）
        # ["256px_ar16:9", "256px_ar4:3", ...]
        bucket_names = sorted(bucket_config.keys(), key=lambda x: aspect_ratios[x][0], reverse=True)

        # バケット名でループして確率とバッチサイズを設定
        for key in bucket_names:
            log_.info(f"{key=}, {aspect_ratios[key]=}")

            # バケットの動画のフレーム数をソート（フレーム数の長い順）
            # [32, 8,...]
            bucket_time_names = sorted(bucket_config[key].keys(), key=lambda x: x, reverse=True)

            # 確率をOrderedDictに格納
            # {32: 0.5, 8: 0.5, ...}
            bucket_probs[key] = OrderedDict({k: bucket_config[key][k][0] for k in bucket_time_names})

            # バッチサイズをOrderedDictに格納
            # {32: 16, 8: 16, ...}
            bucket_bs[key] = OrderedDict({k: bucket_config[key][k][1] for k in bucket_time_names})

        # 2) バケットの基準を設定

        # 解像度の基準
        self.hw_criteria = {k: aspect_ratios[k][0] for k in bucket_names}

        # フレーム数の基準
        self.t_criteria = {k1: {k2: k2 for k2 in bucket_config[k1].keys()} for k1 in bucket_names}

        # アスペクト比の基準
        self.ar_criteria = {
            k1: {k2: {k3: v3 for k3, v3 in aspect_ratios[k1][1].items()} for k2 in bucket_config[k1].keys()}
            for k1 in bucket_names
        }

        bucket_id_cnt = num_bucket = 0
        bucket_id = dict()
        for k1, v1 in bucket_probs.items():
            bucket_id[k1] = dict()
            for k2, _ in v1.items():
                bucket_id[k1][k2] = bucket_id_cnt
                bucket_id_cnt += 1
                num_bucket += len(aspect_ratios[k1][1])

        self.bucket_probs = bucket_probs
        self.bucket_bs = bucket_bs
        self.bucket_id = bucket_id
        self.num_bucket = num_bucket

        log_message("Number of buckets: %s", num_bucket)

    def get_bucket_id(self, T: int, H: int, W: int, fps: float, path: str | None = None, seed: int | None = None, fps_max: int = 16) -> tuple[str, int, int] | None:
        """
        動画のフレーム数、高さ、幅、fpsに基づいて適切なバケットに分類する

        Args:
            T (int): 動画のフレーム数
            H (int): 動画の高さ
            W (int): 動画の幅
            fps (float): 動画のfps
            path (str | None): 動画のパス（デバッグ用、デフォルトはNone）
            seed (int | None): 乱数シード（デフォルトはNone）
            fps_max (int): 最大fps（デフォルトは16）
        Returns:
            tuple[str, int, int] | None:
                適切なバケットのインデックス（バケット名、フレーム数、アスペクト比の名前）のタプル
                適切なバケットが見つからない場合はNoneを返す
        """
        log_.info(f"Bucket.get_bucket_id {T=} {H=} {W=} {fps=} {path=} {seed=} {fps_max=}")
    
        # 許容範囲（80%以上の解像度があれば良い）
        approx = 0.8

        # fpsをmax_fps未満にマッピング
        _, sampling_interval = map_target_fps(fps, fps_max)

        # サンプリング間隔に基づいてフレーム数を調整
        T = T // sampling_interval

        # 動画の解像度を計算
        resolution = H * W

        # 乱数生成器を初期化
        rng = np.random.default_rng(seed)

        # バケットの確率, 高さ・幅の基準, アスペクト比の基準を取得
        bucket_probs = self.bucket_probs
        hw_criteria = self.hw_criteria
        ar_criteria = self.ar_criteria
        log_.debug(f"{bucket_probs=} {hw_criteria=} {ar_criteria=}")

        # バケットを大きい順にループ
        for hw_id, t_criteria in bucket_probs.items():
            log_.debug(f"trying bucket {hw_id=} {t_criteria=}")

            # 動画の解像度がバケットの解像度基準の80%未満の場合、スキップ
            if resolution < hw_criteria[hw_id] * approx:
                continue

            # サンプルが画像の場合
            # False
            if T == 1:
                if 1 in t_criteria:
                    if rng.random() < t_criteria[1]:
                        return hw_id, 1, get_closest_ratio(H, W, ar_criteria[hw_id][1])
                continue

            # 動画のフレーム数が長い順にループ
            for t_id, prob in t_criteria.items():
                log_.debug(f"trying {t_id=} {prob=}")
                if T >= t_id and t_id != 1:
                    # probがタプルの場合、2番目の値を次のt_idにスキップする閾値として使用
                    if isinstance(prob, tuple):
                        next_hw_prob, next_t_prob = prob
                        if next_t_prob >= 1 or rng.random() <= next_t_prob:
                            continue

                    # probがタプルでない場合、そのまま使用
                    else:
                        next_hw_prob = prob

                    # 確率が閾値を超えた場合、バケットを返す
                    if next_hw_prob >= 1 or rng.random() <= next_hw_prob:

                        # 動画のアスペクト比に最も近いアスペクト比を取得
                        ar_id = get_closest_ratio(H, W, ar_criteria[hw_id][t_id])
                        return hw_id, t_id, ar_id

                    # 確率が閾値を超えなかった場合、次のバケットを試す
                    else:
                        break

        return None

    def get_thw(self, bucket_idx: tuple[str, int, int]) -> tuple[int, int, int]:
        assert len(bucket_idx) == 3
        T = self.t_criteria[bucket_idx[0]][bucket_idx[1]]
        H, W = self.ar_criteria[bucket_idx[0]][bucket_idx[1]][bucket_idx[2]]
        return T, H, W

    def get_prob(self, bucket_idx: tuple[str, int]) -> float:
        return self.bucket_probs[bucket_idx[0]][bucket_idx[1]]

    def get_batch_size(self, bucket_idx: tuple[str, int]) -> int:
        return self.bucket_bs[bucket_idx[0]][bucket_idx[1]]

    def __len__(self) -> int:
        return self.num_bucket

# 検証

# MMViTのステージ2のバケット設定
# バケツIDは、AEの圧縮率に合わせて4N+1に設定されている（+1は、最初のフレーム）
stage2_bucket_config = {
    "256px": { 1: (1.0, 130), 5: (1.0, 14), 9: (1.0, 14), 13: (1.0, 14), 17: (1.0, 14), 21: (1.0, 14), 25: (1.0, 14), 29: (1.0, 14), 33: (1.0, 14), 37: (1.0, 10), 41: (1.0, 10), 45: (1.0, 10), 49: (1.0, 10), 53: (1.0, 10), 57: (1.0, 10), 61: (1.0, 10), 65: (1.0, 10), 73: (1.0, 7), 77: (1.0, 7), 81: (1.0, 7), 85: (1.0, 7), 89: (1.0, 7), 93: (1.0, 7), 97: (1.0, 7), 101: (1.0, 6), 105: (1.0, 6), 109: (1.0, 6), 113: (1.0, 6), 117: (1.0, 6), 121: (1.0, 6), 125: (1.0, 6), 129: (1.0, 6), },
    "768px": { 1: (1.0, 38), 5: (1.0, 6), 9: (1.0, 6), 13: (1.0, 6), 17: (1.0, 6), 21: (1.0, 6), 25: (1.0, 6), 29: (1.0, 6), 33: (1.0, 6), 37: (1.0, 4), 41: (1.0, 4), 45: (1.0, 4), 49: (1.0, 4), 53: (1.0, 4), 57: (1.0, 4), 61: (1.0, 4), 65: (1.0, 4), 69: (1.0, 3), 73: (1.0, 3), 77: (1.0, 3), 81: (1.0, 3), 85: (1.0, 3), 89: (1.0, 3), 93: (1.0, 3), 97: (1.0, 3), 101: (1.0, 2), 105: (1.0, 2), 109: (1.0, 2), 113: (1.0, 2), 117: (1.0, 2), 121: (1.0, 2), 125: (1.0, 2), 129: (1.0, 2), },
}

bucket = Bucket(stage2_bucket_config)

# サンプリング間隔2、実質フレーム数30、224*492*0.8は768*768より小さく、256pxバケツに分類
# T=30より小さいIDで最も大きいのは29、従ってバッチサイズ14で訓練
bucket.get_bucket_id(T=60, H=224, W=492, fps=30.0, fps_max=24)

### サンプラーを作成

In [ ]:
# use pandarallel to accelerate bucket processing
# NOTE: pandarallel should only access local variables
def apply(data, method=None, seed=None, num_bucket=None, fps_max=16):
    """
    サンプラーで、データからbucket.get_bucket_idメソッドの引数を作成する
    並列処理でシードを変えるために、動画ごとに異なるシードを使用する
    """
    return method(
        data["num_frames"],
        data["height"],
        data["width"],
        data["fps"],
        data["path"],
        seed + data["id"] * num_bucket, 
        fps_max,
    )

In [ ]:
def sync_object_across_devices(obj: Any, rank: int = 0):
    """
    分散環境のデバイス間でオブジェクトを同期する
    計算したバケツIDを全デバイスで共有するために使用する

    Args:
        obj (Any):
            同期するオブジェクト
            リスト、辞書、カスタムクラスなど、ピクル可能な任意のオブジェクトが使用可能
        rank (int): 
            オブジェクトの状態をブロードキャストするデバイスのランク
            デフォルトは0
    Returns:
        Any: 同期されたオブジェクト
    """

    # オブジェクトをリストに格納
    object_list = [obj]

    # ソースランクから他のすべてのランクにオブジェクトリストをブロードキャスト
    dist.broadcast_object_list(object_list, src=rank, device="cuda")

    # 同期されたオブジェクトを取得
    obj = object_list[0]

    return obj

sync_object_across_devices("test")

In [ ]:
def format_numel_str(numel: int) -> str:
    """
    数値を人間が読みやすい形式の文字列に変換する
    例えば、12345678 -> '11.77 M'など
    バケツの統計データを表示するために使用する

    Args:
        numel (int): 要素数
    Returns:
        str: フォーマットされた文字列
    """
    B = 1024**3 # billion
    M = 1024**2 # million
    K = 1024 # thousand

    if numel >= B:
        return f"{numel / B:.2f} B"
    elif numel >= M:
        return f"{numel / M:.2f} M"
    elif numel >= K:
        return f"{numel / K:.2f} K"
    else:
        return f"{numel}"

format_numel_str(12345678)  # '11.77 M'

In [ ]:
class VariableVideoBatchSampler(DistributedSampler):
    """
    動画データセットのための可変バッチサイズサンプラー
    データローダーのサンプラーとして使用される
    バケツに基づいて動画をグループ化し、各バケツに対して異なるバッチサイズでサンプリングする
    PyTorchのDistributedSamplerを継承し、各プロセスが重複なく均等にサンプリングできる
    """

    def __init__(
        self,
        dataset: VideoTextDataset,
        bucket_config: dict,
        num_replicas: int | None = None,
        rank: int | None = None,
        shuffle: bool = True,
        seed: int = 0,
        drop_last: bool = False,
        verbose: bool = False,
        num_bucket_build_workers: int = 1,
        num_groups: int = 1,
    ) -> None:

        # DistributedSamplerの初期化
        super().__init__(
            dataset=dataset, num_replicas=num_replicas, rank=rank, shuffle=shuffle, seed=seed, drop_last=drop_last
        )
        self.dataset = dataset
        assert dataset.bucket_class == "Bucket", "Only support Bucket class for now"

        # バケツを初期化
        self.bucket = Bucket(bucket_config)
        self.verbose = verbose
        self.last_micro_batch_access_index = 0
        self.num_bucket_build_workers = num_bucket_build_workers
        self._cached_bucket_sample_dict = None
        self._cached_num_total_batch = None
        self.num_groups = num_groups

        if dist.get_rank() == 0:
            # pandarallelの初期化
            # 大量の動画をバケツに分類する処理を分散する
            pandarallel.initialize(
                nb_workers=self.num_bucket_build_workers,
                progress_bar=False,
                verbose=0,
                use_memory_fs=False,
            )

    def __iter__(self) -> Iterator[list[int]]:
        """
        バケツに基づいてデータセットからサンプルをイテレートする
        """

        # 1) グループ化されたバケツサンプルの辞書を取得
        bucket_sample_dict, _ = self.group_by_bucket()
        log_.debug(f"Bucket sample dict keys: {list(bucket_sample_dict.keys())}")

        self.clear_cache()

        g = torch.Generator()
        g.manual_seed(self.seed + self.epoch)
        bucket_micro_batch_count = OrderedDict()
        bucket_last_consumed = OrderedDict()

        # 2) 各バケツの中でバッチを作る

        for bucket_id, data_list in bucket_sample_dict.items():
            log_.debug(f"Processing bucket {bucket_id} with {len(data_list)} samples")

            # バッチサイズを取得
            bs_per_gpu = self.bucket.get_batch_size(bucket_id)
            remainder = len(data_list) % bs_per_gpu

            if remainder > 0:
                if not self.drop_last:
                    # if there is remainder, we pad to make it divisible
                    data_list += data_list[: bs_per_gpu - remainder]
                else:
                    # we just drop the remainder to make it divisible
                    data_list = data_list[:-remainder]
            bucket_sample_dict[bucket_id] = data_list

            # バケツ内でシャッフル
            if self.shuffle:
                data_indices = torch.randperm(len(data_list), generator=g).tolist()
                data_list = [data_list[i] for i in data_indices]
                bucket_sample_dict[bucket_id] = data_list

            # バケツから得られるマイクロバッチの数を計算
            num_micro_batches = len(data_list) // bs_per_gpu
            bucket_micro_batch_count[bucket_id] = num_micro_batches

        # 3) バケツのアクセス順序を決めるリストを作る
        # 例: [bucket_id_1, bucket_id_1, bucket_id_2, bucket_id_1, ...]

        bucket_id_access_order = []

        for bucket_id, num_micro_batch in bucket_micro_batch_count.items():
            bucket_id_access_order.extend([bucket_id] * num_micro_batch)

        # 4) バケツの順番をシャッフル

        if self.shuffle:
            bucket_id_access_order_indices = torch.randperm(len(bucket_id_access_order), generator=g).tolist()
            bucket_id_access_order = [bucket_id_access_order[i] for i in bucket_id_access_order_indices]

        # make the number of bucket accesses divisible by dp size
        remainder = len(bucket_id_access_order) % self.num_replicas
        if remainder > 0:
            if self.drop_last:
                bucket_id_access_order = bucket_id_access_order[: len(bucket_id_access_order) - remainder]
            else:
                bucket_id_access_order += bucket_id_access_order[: self.num_replicas - remainder]

        # 5) 分散環境で、バケツからバッチを準備

        num_iters = len(bucket_id_access_order) // self.num_replicas
        start_iter_idx = self.last_micro_batch_access_index // self.num_replicas

        # re-compute the micro-batch consumption
        # this is useful when resuming from a state dict with a different number of GPUs
        self.last_micro_batch_access_index = start_iter_idx * self.num_replicas
        for i in range(self.last_micro_batch_access_index):
            bucket_id = bucket_id_access_order[i]
            bucket_bs = self.bucket.get_batch_size(bucket_id)
            if bucket_id in bucket_last_consumed:
                bucket_last_consumed[bucket_id] += bucket_bs
            else:
                bucket_last_consumed[bucket_id] = bucket_bs

        # 6) バッチを一つ取り出してyieldする

        for i in range(start_iter_idx, num_iters):

            log_.debug(f"Sampling micro-batch {i+1}/{num_iters}")

            # 
            bucket_access_list = bucket_id_access_order[i * self.num_replicas : (i + 1) * self.num_replicas]
            self.last_micro_batch_access_index += self.num_replicas

            # compute the data samples consumed by each access
            bucket_access_boundaries = []
            for bucket_id in bucket_access_list:
                bucket_bs = self.bucket.get_batch_size(bucket_id)
                last_consumed_index = bucket_last_consumed.get(bucket_id, 0)
                bucket_access_boundaries.append([last_consumed_index, last_consumed_index + bucket_bs])

                # update consumption
                if bucket_id in bucket_last_consumed:
                    bucket_last_consumed[bucket_id] += bucket_bs
                else:
                    bucket_last_consumed[bucket_id] = bucket_bs

            # GPUごとにアクセスされるデータの範囲を計算
            bucket_id = bucket_access_list[self.rank]
            boundary = bucket_access_boundaries[self.rank]
            cur_micro_batch = bucket_sample_dict[bucket_id][boundary[0] : boundary[1]]

            # サンプルインデックスにt, h, wをエンコード
            real_t, real_h, real_w = self.bucket.get_thw(bucket_id)

            # idx-t-h-wの形式に変換
            cur_micro_batch = [f"{idx}-{real_t}-{real_h}-{real_w}" for idx in cur_micro_batch]

            # 現在のマイクロバッチを返す
            yield cur_micro_batch

        self.reset()

    def __len__(self) -> int:
        return self.get_num_batch() // self.num_groups

    def get_num_batch(self) -> int:
        _, num_total_batch = self.group_by_bucket()
        return num_total_batch

    def clear_cache(self):
        self._cached_bucket_sample_dict = None
        self._cached_num_total_batch = 0

    def group_by_bucket(self) -> dict:
        """
        データセットのサンプルをバケツにグループ化する
        __iter__メソッドの最初に呼び出される

        Returns:
            dict: バケツIDをキー、サンプルインデックスのリストを値とする辞書
        """

        # すでにキャッシュされている場合、キャッシュを返す
        if self._cached_bucket_sample_dict is not None:
            return self._cached_bucket_sample_dict, self._cached_num_total_batch

        log_message("Building buckets using %d workers...", self.num_bucket_build_workers)

        # バケツIDを計算
        bucket_ids = None
        if dist.get_rank() == 0:
            data = self.dataset.data.copy(deep=True)
            data["id"] = data.index
            bucket_ids = data.parallel_apply(
                apply,
                axis=1,
                method=self.bucket.get_bucket_id,
                seed=self.seed + self.epoch,
                num_bucket=self.bucket.num_bucket,
                fps_max=self.dataset.fps_max,
            )

        # すべてのデバイスでバケツIDを同期
        dist.barrier()
        bucket_ids = sync_object_across_devices(bucket_ids)
        log_.debug(f"{bucket_ids=}")
        dist.barrier()

        # バケツごとに動画のインデックスのリストを格納
        bucket_sample_dict = defaultdict(list)
        bucket_ids_np = np.array(bucket_ids)
        valid_indices = np.where(bucket_ids_np != None)[0]
        log_.debug(f"{valid_indices=}")

        for i in valid_indices:
            bucket_sample_dict[bucket_ids_np[i]].append(i)

        # バケツサンプル辞書をキャッシュ
        self._cached_bucket_sample_dict = bucket_sample_dict

        # すべてのバケツのバッチ数を計算
        num_total_batch = self.print_bucket_info(bucket_sample_dict)
        self._cached_num_total_batch = num_total_batch

        return bucket_sample_dict, num_total_batch

    def print_bucket_info(self, bucket_sample_dict: dict) -> int:
        """
        バケツの統計情報を収集してログに記録する
        group_by_bucketメソッドで呼び出される

        Args:
            bucket_sample_dict (dict):
                バケツIDをキー、サンプルインデックスのリストを値とする辞書
        Returns:
            int: 総バッチ数
        """
        # collect statistics
        num_total_samples = num_total_batch = 0
        num_total_img_samples = num_total_vid_samples = 0
        num_total_img_batch = num_total_vid_batch = 0
        num_total_vid_batch_256 = num_total_vid_batch_768 = 0
        num_aspect_dict = defaultdict(lambda: [0, 0])
        num_hwt_dict = defaultdict(lambda: [0, 0])
        for k, v in bucket_sample_dict.items():
            size = len(v)
            num_batch = size // self.bucket.get_batch_size(k[:-1])

            num_total_samples += size
            num_total_batch += num_batch

            if k[1] == 1:
                num_total_img_samples += size
                num_total_img_batch += num_batch
            else:
                if k[0] == "256px":
                    num_total_vid_batch_256 += num_batch
                elif k[0] == "768px":
                    num_total_vid_batch_768 += num_batch
                num_total_vid_samples += size
                num_total_vid_batch += num_batch

            num_aspect_dict[k[-1]][0] += size
            num_aspect_dict[k[-1]][1] += num_batch
            num_hwt_dict[k[:-1]][0] += size
            num_hwt_dict[k[:-1]][1] += num_batch

        # sort
        num_aspect_dict = dict(sorted(num_aspect_dict.items(), key=lambda x: x[0]))
        num_hwt_dict = dict(
            sorted(num_hwt_dict.items(), key=lambda x: (get_num_pexels_from_name(x[0][0]), x[0][1]), reverse=True)
        )
        num_hwt_img_dict = {k: v for k, v in num_hwt_dict.items() if k[1] == 1}
        num_hwt_vid_dict = {k: v for k, v in num_hwt_dict.items() if k[1] > 1}

        # log
        if dist.get_rank() == 0 and self.verbose:
            log_message("Bucket Info:")
            log_message("Bucket [#sample, #batch] by aspect ratio:")
            for k, v in num_aspect_dict.items():
                log_message("(%s): #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("===== Image Info =====")
            log_message("Image Bucket by HxWxT:")
            for k, v in num_hwt_img_dict.items():
                log_message("%s: #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("--------------------------------")
            log_message(
                "#image sample: %s, #image batch: %s",
                format_numel_str(num_total_img_samples),
                format_numel_str(num_total_img_batch),
            )
            log_message("===== Video Info =====")
            log_message("Video Bucket by HxWxT:")
            for k, v in num_hwt_vid_dict.items():
                log_message("%s: #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("--------------------------------")
            log_message(
                "#video sample: %s, #video batch: %s",
                format_numel_str(num_total_vid_samples),
                format_numel_str(num_total_vid_batch),
            )
            log_message("===== Summary =====")
            log_message("#non-empty buckets: %s", len(bucket_sample_dict))
            log_message(
                "Img/Vid sample ratio: %.2f",
                num_total_img_samples / num_total_vid_samples if num_total_vid_samples > 0 else 0,
            )
            log_message(
                "Img/Vid batch ratio: %.2f", num_total_img_batch / num_total_vid_batch if num_total_vid_batch > 0 else 0
            )
            log_message(
                "vid batch 256: %s, vid batch 768: %s", format_numel_str(num_total_vid_batch_256), format_numel_str(num_total_vid_batch_768)
            )
            log_message(
                "Vid batch ratio (256px/768px): %.2f", num_total_vid_batch_256 / num_total_vid_batch_768 if num_total_vid_batch_768 > 0 else 0
            )
            log_message(
                "#training sample: %s, #training batch: %s",
                format_numel_str(num_total_samples),
                format_numel_str(num_total_batch),
            )
        return num_total_batch

    def reset(self):
        self.last_micro_batch_access_index = 0

    def set_step(self, start_step: int):
        self.last_micro_batch_access_index = start_step * self.num_replicas

    def state_dict(self, num_steps: int) -> dict:
        # the last_micro_batch_access_index in the __iter__ is often
        # not accurate during multi-workers and data prefetching
        # thus, we need the user to pass the actual steps which have been executed
        # to calculate the correct last_micro_batch_access_index
        return {"seed": self.seed, "epoch": self.epoch, "last_micro_batch_access_index": num_steps * self.num_replicas}

    def load_state_dict(self, state_dict: dict) -> None:
        self.__dict__.update(state_dict)

In [ ]:
def collate_fn_default(batch):
    """
    データローダのデフォルトの整形関数
    """

    # 1) バッチからNoneをフィルタリング
    batch = [x for x in batch if x is not None]

    assert len(batch) > 0, "batch is empty"

    # 2) テキスト特徴量をバッチから分離

    use_mask = False
    if "mask" in batch[0] and isinstance(batch[0]["mask"], int):

        # テキストマスクをバッチから分離
        masks = [x.pop("mask") for x in batch]

        # テキストをバッチから分離
        texts = [x.pop("text") for x in batch]

        # バッチ次元で手動で連結
        texts = torch.cat(texts, dim=1)
        use_mask = True

    # 3) テキスト特徴量を除いたバッチをデフォルトのcollate関数で整形

    # バッチ次元を持つテンソルに変換
    ret = torch.utils.data.default_collate(batch)

    # 4) テキストデータを戻す

    if use_mask:
        ret["mask"] = masks
        ret["text"] = texts
    return ret

In [ ]:
def get_seed_worker(seed):
    """
    データローダの再現性を確保するシードワーカーを取得する関数
    分散環境で訓練の再現性を確保するために使用する
    """

    # シードワーカー関数をシード値で初期化して返す
    def seed_worker(worker_id):
        worker_seed = seed
        if seed is not None:
            np.random.seed(worker_seed)
            torch.manual_seed(worker_seed)
            random.seed(worker_seed)

    return seed_worker

In [ ]:
def prepare_dataloader(dataset, batch_size=None, shuffle=False, seed=1024, drop_last=False, pin_memory=False, num_workers=0, process_group: ProcessGroup | None = None, bucket_config=None, num_bucket_build_workers=1, prefetch_factor=None, cache_pin_memory=False, num_groups=1, **kwargs):
    """
    データセットに基づいてデータローダを準備する関数

    Args:
        dataset (VideoTextDataset | TextDataset):
            データローダを準備するためのデータセット
        batch_size (int, optional):
            テキストデータセット用のバッチサイズ
            動画データセットでは、バケツに基づいて可変バッチサイズが使用されるため、無視される
            デフォルトはNone
        shuffle (bool, optional):
            データセットをシャッフルするかどうか
            デフォルトはFalse
        seed (int, optional):
            再現性のためのシード値
            デフォルトは1024
        drop_last (bool, optional):
            バッチが不完全な場合に最後のバッチをドロップするかどうか
            デフォルトはFalse
        pin_memory (bool, optional):
            データローダがピンメモリを使用するかどうか
            デフォルトはFalse
        num_workers (int, optional):
            データローダのワーカープロセス数
            デフォルトは0
        process_group (ProcessGroup | None, optional):
            分散訓練のためのプロセスグループ
            Noneの場合、分散サンプリングは使用されない
            デフォルトはNone
        bucket_config (dict, optional):
            動画データセット用のバケツ設定
            動画データセットでのみ使用される
            デフォルトはNone
        num_bucket_build_workers (int, optional):
            バケツ構築のためのワーカープロセス数
            デフォルトは1
        prefetch_factor (int, optional):
            各ワーカーが事前にフェッチするバッチ数
            デフォルトはNone
        cache_pin_memory (bool, optional):
            動画データセット用のキャッシュピンメモリを使用するかどうか
            デフォルトはFalse
        num_groups (int, optional):
            可変バッチサンプラーのグループ数
            デフォルトは1
        **kwargs:
            DataLoaderに渡される追加のキーワード引数
    Returns:
        tuple:
            DataLoaderオブジェクトとサンプラーオブジェクトのタプル  
    """
    log_.info(f"prepare_dataloader: {dataset=} {batch_size=} {shuffle=} {seed=} {drop_last=} {pin_memory=} {num_workers=} {process_group=} {bucket_config=} {num_bucket_build_workers=} {prefetch_factor=} {cache_pin_memory=} {num_groups=}")

    _kwargs = kwargs.copy()

    # True
    if isinstance(dataset, VideoTextDataset):

        # 動画データセット用の可変バッチサンプラーを作成
        batch_sampler = VariableVideoBatchSampler(
            dataset,
            bucket_config,
            num_replicas=process_group.size(),
            rank=process_group.rank(),
            shuffle=shuffle,
            seed=seed,
            drop_last=drop_last,
            verbose=True,
            num_bucket_build_workers=num_bucket_build_workers,
            num_groups=num_groups,
        )

        # DataLoaderクラスを選択
        dl_cls = DataloaderForVideo if cache_pin_memory else DataLoader

        # データローダとサンプラーを返す
        return (
            dl_cls(
                dataset,
                batch_sampler=batch_sampler,
                worker_init_fn=get_seed_worker(seed),
                pin_memory=pin_memory,
                num_workers=num_workers,
                collate_fn=collate_fn_default,
                prefetch_factor=prefetch_factor,
                **_kwargs,
            ),
            batch_sampler,
        )
    elif isinstance(dataset, TextDataset):
        if process_group is None:
            return (
                DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    worker_init_fn=get_seed_worker(seed),
                    drop_last=drop_last,
                    pin_memory=pin_memory,
                    num_workers=num_workers,
                    prefetch_factor=prefetch_factor,
                    **_kwargs,
                ),
                None,
            )
        else:
            sampler = DistributedSampler(
                dataset,
                num_replicas=process_group.size(),
                rank=process_group.rank(),
                shuffle=shuffle,
                seed=seed,
                drop_last=drop_last,
            )
            return (
                DataLoader(
                    dataset,
                    sampler=sampler,
                    worker_init_fn=get_seed_worker(seed),
                    pin_memory=pin_memory,
                    num_workers=num_workers,
                    collate_fn=collate_fn_default,
                    prefetch_factor=prefetch_factor,
                    **_kwargs,
                ),
                sampler,
            )
    else:
        raise ValueError(f"Unsupported dataset type: {type(dataset)}")

In [ ]:
def get_data_parallel_group(get_mixed_dp_pg : bool = False):
    """
    データ並列グループをグローバル変数から取得する

    Args:
        get_mixed_dp_pg (bool):
            混合データ並列グループを取得するかどうか
            混合データ並列は、データ並列と他の並列手法を混合した手法
    Returns:
        dist.ProcessGroup: データ並列グループ
    """

    # 混合データ並列グループが要求され、グローバル変数に存在する場合
    if get_mixed_dp_pg and "mixed_dp_group" in _GLOBAL_PARALLEL_GROUPS:
        # 混合データ並列グループを返す
        return _GLOBAL_PARALLEL_GROUPS["mixed_dp_group"]

    # 通常のデータ並列グループを返す
    return _GLOBAL_PARALLEL_GROUPS.get("data", dist.group.WORLD)

get_data_parallel_group()

In [ ]:
# データローダの引数を設定

cache_pin_memory = pin_memory_cache_pre_alloc_numels is not None

dataloader_args = dict(
    dataset=dataset,
    batch_size=cfg.get("batch_size", None),
    num_workers=cfg.get("num_workers", 4),
    seed=cfg.get("seed", 1024),
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    process_group=get_data_parallel_group(),
    prefetch_factor=cfg.get("prefetch_factor", None),
    cache_pin_memory=cache_pin_memory,
)

dataloader_args

In [ ]:
# データローダとサンプラーを準備
dataloader, sampler = prepare_dataloader(
    bucket_config=cfg.get("bucket_config", None),
    num_bucket_build_workers=cfg.get("num_bucket_build_workers", 1),
    **dataloader_args,
)
dataloader, sampler

In [ ]:
first_batch = next(iter(dataloader))
first_batch["sampling_interval"], first_batch["video"].shape, first_batch["path"], first_batch["index"]

In [ ]:
first_frame = first_batch["video"][:, :, 0].to(device=device, dtype=torch.float32)
first_frame.shape

import matplotlib.pyplot as plt
plt.imshow(first_frame[0].permute(1, 2, 0).cpu().numpy())
plt.axis("off")
plt.show()

### モデルの初期化

In [ ]:
cfg.model

In [ ]:
# @MODELS.register_module("dc_ae")
def DC_AE(
    model_name: str,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    from_scratch: bool = False,
    from_pretrained: str | None = None,
    is_training: bool = False,
    use_spatial_tiling: bool = False,
    use_temporal_tiling: bool = False,
    spatial_tile_size: int = 256,
    temporal_tile_size: int = 32,
    tile_overlap_factor: float = 0.25,
    scaling_factor: float = None,
    disc_off_grad_ckpt: bool = False,
) -> DCAE_HF:
    if not from_scratch:
        model = DCAE_HF.from_pretrained(model_name).to(device_map, torch_dtype)
    else:
        model = DCAE_HF(model_name).to(device_map, torch_dtype)

    if from_pretrained is not None:
        model = load_checkpoint(model, from_pretrained, device_map=device_map)
        print(f"loaded dc_ae from ckpt path: {from_pretrained}")

    model.cfg.is_training = is_training
    model.use_spatial_tiling = use_spatial_tiling
    model.use_temporal_tiling = use_temporal_tiling
    model.spatial_tile_size = spatial_tile_size
    model.temporal_tile_size = temporal_tile_size
    model.tile_overlap_factor = tile_overlap_factor
    if scaling_factor is not None:
        model.scaling_factor = scaling_factor
    model.decoder.disc_off_grad_ckpt = disc_off_grad_ckpt
    return model

# {'type': 'dc_ae',
#  'model_name': 'dc-ae-f32t4c128',
#  'from_scratch': True,
#  'from_pretrained': None}

model = DC_AE(
    model_name=cfg.model.model_name,
    from_scratch=cfg.model.get("from_scratch", False),
    from_pretrained=cfg.model.get("from_pretrained", None),
)
model

### モデルのパラメータ数を記録

In [ ]:
def get_model_numel(model: torch.nn.Module) -> tuple[int, int]:
    """
    ログようにモデルのパラメータ数を取得する関数

    Args:
        model (torch.nn.Module): 対象のモデル

    Returns:
        tuple[int, int]: モデルの総パラメータ数と学習可能なパラメータ数のタプル
    """
    num_params = 0
    num_params_trainable = 0

    # モデルの各パラメータをループ
    for p in model.parameters():

        # 総パラメータ数を加算
        num_params += p.numel()

        # 学習可能なパラメータ数を加算
        if p.requires_grad:
            num_params_trainable += p.numel()

    # 総パラメータ数と学習可能なパラメータ数を返す
    return num_params, num_params_trainable

In [ ]:
def log_model_params(model: nn.Module):
    """
    モデルのパラメータ数をログに記録する

    Args:
        model (torch.nn.Module): 対象のモデル
    """

    # 総パラメータ数と学習可能なパラメータ数を取得
    num_params, num_params_trainable = get_model_numel(model)

    # モデルのクラス名を取得
    model_name = model.__class__.__name__

    # ログに記録
    log_message(f"[{model_name}] Number of parameters: {format_numel_str(num_params)}")
    log_message(f"[{model_name}] Number of trainable parameters: {format_numel_str(num_params_trainable)}")

log_model_params(model)

In [ ]:
# 勾配チェックポインティングを設定
# False
if cfg.get("grad_checkpoint", False):
    set_grad_checkpoint(model)

### LPIPSの作成

LPIPS = Learned Perceptual Image Patch Similarity

人間の目から見て、2つの画像がどのくらい似ているかを測るための評価指標

画像認識モデルVGG16に正解のフレームと生成したフレームの類似度を計測する

In [ ]:
class vgg16(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True):
        """
        Args:
            requires_grad (bool): パラメータの勾配計算を行うかどうか
            pretrained (bool): 事前学習済みモデルを使用するかどうか
        """
        super(vgg16, self).__init__()

        # 1) 事前学習済みVGG16モデルの特徴抽出部分を取得        
        vgg_pretrained_features = models.vgg16(pretrained=pretrained).features

        # 2) VGG16の層を5つのスライスに分割

        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.N_slices = 5

        # 0~3層
        # エッジなどの低レベル特徴を抽出
        for x in range(4):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])

        # 4~8層
        for x in range(4, 9):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])

        # 9~15層
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])

        # 16~22層
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])

        # 23~29層
        # 物体の一部などの高レベル特徴を抽出
        for x in range(23, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])

        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        """
        vgg16モデルの順伝播を実行し、特定の層の出力を取得する
        """

        # 1) 順伝搬を実行し、各スライスの出力を取得

        h = self.slice1(X)
        h_relu1_2 = h

        h = self.slice2(h)
        h_relu2_2 = h

        h = self.slice3(h)
        h_relu3_3 = h

        h = self.slice4(h)
        h_relu4_3 = h

        h = self.slice5(h)
        h_relu5_3 = h

        # 2) namedtupleを使用して出力を整理

        vgg_outputs = namedtuple("VggOutputs", [
            "relu1_2",
            "relu2_2",
            "relu3_3",
            "relu4_3",
            "relu5_3"
        ])

        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3, h_relu5_3)
        return out

In [ ]:
class NetLinLayer(nn.Module):
    """
    VGGの特徴マップに適用される線形層
    1x1の畳み込み層として実装され、チャネル数を調整する
    画像の知覚的損失を計算するために使用される
    """

    def __init__(self, chn_in, chn_out=1, use_dropout=False):
        super(NetLinLayer, self).__init__()

        # ドロップアウト層をオプションで追加
        layers = ([ nn.Dropout() ] if (use_dropout) else [])

        # 1x1の畳み込み層を追加
        # チャンネル数をchn_inからchn_outに削減
        layers += [
            nn.Conv2d(
                chn_in,
                chn_out,
                1,
                stride=1,
                padding=0,
                bias=False, # バイアスなし
            ),
        ]

        self.model = nn.Sequential(*layers)

In [ ]:
def download(url, local_path, chunk_size=1024):
    """
    URLからファイルをダウンロードし、ローカルパスに保存する関数
    LPIPSモデルの重みファイルをダウンロードするために使用される
    """

    # ダウンロードディレクトリを作成
    os.makedirs(os.path.split(local_path)[0], exist_ok=True)

    # ストリーミングモードでHTTP GETリクエストを送信
    with requests.get(url, stream=True) as r:

        # レスポンスヘッダからコンテンツの総サイズを取得
        total_size = int(r.headers.get("content-length", 0))

        # 進捗バーを表示しながらデータをチャンクごとに書き込む
        with tqdm(total=total_size, unit="B", unit_scale=True) as pbar:
            with open(local_path, "wb") as f:
                for data in r.iter_content(chunk_size=chunk_size):
                    if data:
                        f.write(data)
                        pbar.update(chunk_size)

In [ ]:
def md5_hash(path):
    with open(path, "rb") as f:
        content = f.read()
    return hashlib.md5(content).hexdigest()

In [ ]:
# チェックポイントファイル名のマッピング
CKPT_MAP = {"vgg_lpips": "vgg.pth"}

# MD5チェック用のハッシュ値
MD5_MAP = {"vgg_lpips": "d507d7349b931f0638a25a48a722f98a"}

# 事前学習済みモデルのダウンロードURLマッピング
URL_MAP = {"vgg_lpips": "https://heibox.uni-heidelberg.de/f/607503859c864bc1b30b/?dl=1"}


def get_ckpt_path(name, root=".", check=False):
    """
    事前学習済みモデルのチェックポイントパスを取得する関数
    """

    assert name in URL_MAP

    # チェックポイントの保存パスを作成
    path = os.path.join(root, CKPT_MAP[name])

    # チェックポイントが存在しないか、MD5チェックが有効で不一致の場合、ダウンロードする
    if not os.path.exists(path) or (check and not md5_hash(path) == MD5_MAP[name]):
        print("Downloading {} model from {} to {}".format(name, URL_MAP[name], path))
        download(URL_MAP[name], path)
        md5 = md5_hash(path)
        assert md5 == MD5_MAP[name], md5
    return path

In [ ]:
@torch._disable_dynamo
def checkpoint(function, *args, use_reentrant: Optional[bool] = None, context_fn: Callable[[], Tuple[ContextManager, ContextManager]] = noop_context_fn, determinism_check: str = _DEFAULT_DETERMINISM_MODE, debug: bool = False, **kwargs):
    """
    アクティベーションチェックポインティングを実行する関数
    """
    if use_reentrant is None:
        warnings.warn(
            "torch.utils.checkpoint: the use_reentrant parameter should be "
            "passed explicitly. In version 2.4 we will raise an exception "
            "if use_reentrant is not passed. use_reentrant=False is "
            "recommended, but if you need to preserve the current default "
            "behavior, you can pass use_reentrant=True. Refer to docs for more "
            "details on the differences between the two variants.",
            stacklevel=2,
        )
        use_reentrant = True

    # Hack to mix *args with **kwargs in a python 2.7-compliant way
    preserve = kwargs.pop("preserve_rng_state", True)
    if kwargs and use_reentrant:
        raise ValueError("Unexpected keyword arguments: " + ",".join(arg for arg in kwargs))

    if use_reentrant:
        if context_fn is not noop_context_fn or debug is not False:
            raise ValueError("Passing `context_fn` or `debug` is only supported when " "use_reentrant=False.")
        return CheckpointFunctionWithOffload.apply(function, preserve, *args)
    else:
        gen = _checkpoint_without_reentrant_generator(
            function, preserve, context_fn, determinism_check, debug, *args, **kwargs
        )
        # Runs pre-forward logic
        next(gen)
        ret = function(*args, **kwargs)
        # Runs post-forward logic
        try:
            next(gen)
        except StopIteration:
            return ret

In [ ]:
def normalize_tensor(x, eps=1e-10):
    """
    テンソルをチャンネルごとにL2正規化する関数
    各サンプルのL2ノルムを1にスケーリングする
    LPIPSの損失計算で使用

    x = x / (||x||_2 + eps)
    """
    # (B, C, H, W)のテンソルを想定
    norm_factor = torch.sqrt(torch.sum(x**2, dim=1, keepdim=True))
    return x / (norm_factor + eps)

In [ ]:
def spatial_average(x, keepdim=True):
    """
    空間次元（高さと幅）にわたるテンソルの平均を計算する関数
    LPIPSの損失計算で、ピクセルごとの損失を集約するために使用
    """

    # (B, C, H, W)のテンソルを想定
    return x.mean([2, 3], keepdim=keepdim)

In [ ]:
class ScalingLayer(nn.Module):
    """
    入力画像をLPIPSモデルの期待する範囲にスケーリングする層
    入力画像は[-1, 1]の範囲に正規化されていると仮定
    LPIPSモデルは、[-1.030, 0.920]の範囲で動作するように設計されている
    LPIPSの入力時に使用される
    """

    def __init__(self):
        super(ScalingLayer, self).__init__()
        self.register_buffer("shift", torch.Tensor([-0.030, -0.088, -0.188])[None, :, None, None])
        self.register_buffer("scale", torch.Tensor([0.458, 0.448, 0.450])[None, :, None, None])

    def forward(self, inp):
        return (inp - self.shift) / self.scale

In [ ]:
class LPIPS(nn.Module):
    """
    Learned Perceptual Image Patch Similarity (LPIPS) モデル
    画像の知覚的類似度を測定するために使用される
    """

    def __init__(self, use_dropout=True):
        super().__init__()

        # 入力のスケーリング層
        self.scaling_layer = ScalingLayer()

        # VGG16のチャンネル数
        self.chns = [64, 128, 256, 512, 512]

        # 事前学習済みVGG16モデルをロード
        self.net = vgg16(pretrained=True, requires_grad=False)

        # 各VGG16の特徴マップに対応する線形層を作成
        # 出力チャネル数を1にする
        self.lin0 = NetLinLayer(self.chns[0], use_dropout=use_dropout)
        self.lin1 = NetLinLayer(self.chns[1], use_dropout=use_dropout)
        self.lin2 = NetLinLayer(self.chns[2], use_dropout=use_dropout)
        self.lin3 = NetLinLayer(self.chns[3], use_dropout=use_dropout)
        self.lin4 = NetLinLayer(self.chns[4], use_dropout=use_dropout)

        # 線形層をリストにまとめる
        self.lins = [self.lin0, self.lin1, self.lin2, self.lin3, self.lin4]

        # 事前学習済みのLPIPSモデルの重みをロード
        self.load_from_pretrained()

        # すべてのパラメータの勾配計算を無効化
        for param in self.parameters():
            param.requires_grad = False

    def load_from_pretrained(self, name="vgg_lpips"):
        """
        事前学習済みのLPIPSモデルの重みをロードする
        """
        path = os.path.expanduser("~/.cache/opensora/taming/modules/autoencoder/lpips")
        ckpt = get_ckpt_path(name, path)
        self.load_state_dict(torch.load(ckpt, map_location=torch.device("cpu")), strict=False)

    @classmethod
    def from_pretrained(cls, name="vgg_lpips"):
        if name != "vgg_lpips":
            raise NotImplementedError
        model = cls()
        ckpt = get_ckpt_path(name)
        model.load_state_dict(torch.load(ckpt, map_location=torch.device("cpu")), strict=False)
        return model

    def forward_old(self, input, target):
        in0_input, in1_input = (self.scaling_layer(input), self.scaling_layer(target))
        outs0, outs1 = self.net(in0_input), self.net(in1_input)
        feats0, feats1, diffs = {}, {}, {}
        lins = [self.lin0, self.lin1, self.lin2, self.lin3, self.lin4]
        for kk in range(len(self.chns)):
            feats0[kk], feats1[kk] = normalize_tensor(outs0[kk]), normalize_tensor(outs1[kk])
            diffs[kk] = (feats0[kk] - feats1[kk]) ** 2

        res = [spatial_average(lins[kk].model(diffs[kk]), keepdim=True) for kk in range(len(self.chns))]
        val = res[0]
        for l in range(1, len(self.chns)):
            val += res[l]
        return val

    def get_layer_loss(self, input, target, i):
        """
        指定された層でのLPIPS損失を計算する関数

        Args:
            input (torch.Tensor): 入力画像のテンソル
            target (torch.Tensor): ターゲット画像のテンソル
            i (int): 計算する層のインデックス
        Returns:
            tuple: 平均損失、入力特徴マップ、ターゲット特徴マップ
        """
        log_.info(f"LPIPS get_layer_loss: {input.shape=} {target.shape=} {i=}")

        # 指定された層の順伝搬を、入力画像とターゲット画像に対して実行
        input, target = getattr(self.net, f"slice{i+1}")(input), getattr(self.net, f"slice{i+1}")(target)

        # 特徴マップを正規化
        feats0, feats1 = normalize_tensor(input), normalize_tensor(target)

        # 特徴マップの差の二乗を計算
        diff = (feats0 - feats1) ** 2

        # 空間平均を計算して平均損失を取得
        avg = spatial_average(self.lins[i].model(diff), keepdim=True)

        return avg, input, target

    def forward(self, input, target):
        """
        LPIPSモデルの順伝播を実行し、知覚的損失を計算する
        """
        log_.info(f"LPIPS forward: {input.shape=} {target.shape=}")

        # 入力とターゲットをスケーリング
        input, target = (self.scaling_layer(input), self.scaling_layer(target))

        val = None
        for i in range(len(self.chns)):
            # アクティベーションチェックポイントを使用して層ごとの損失を計算
            avg, input, target = checkpoint(
                self.get_layer_loss,
                input,
                target,
                i,
                use_reentrant=False
            )

            # 損失を累積
            val = avg if val is None else val + avg

        return val

### VAELossの作成

VAELossは、元の動画と復元した動画を比較する評価指標:

- 再構成損失（L1損失）: ピクセルごとの色のずれ `recon_loss`
- 知覚損失（LPIPS）: 人が見てクッキリしているか `perceptual_loss`
- KL損失: 潜在表現が綺麗なガウス分布に近いかどうか `kl_loss`

In [ ]:
def l1(x, y):
    """
    L1損失関数
    2つのテンソル間の絶対差の要素ごとの和を計算する
    """
    return torch.abs(x - y)

In [ ]:
def batch_mean(x):
    """
    バッチ次元にわたるテンソルの平均を計算する関数
    """
    return torch.sum(x) / x.shape[0]

In [ ]:
class VAELoss(nn.Module):
    def __init__(
        self,
        logvar_init=0.0,
        perceptual_loss_weight=1.0,
        kl_loss_weight=5e-4,
        device="cpu",
        dtype="bf16",
    ):
        super().__init__()

        if type(dtype) == str:
            if dtype == "bf16":
                dtype = torch.bfloat16
            elif dtype == "fp16":
                dtype = torch.float16
            elif dtype == "fp32":
                dtype = torch.float32
            else:
                raise NotImplementedError(f"dtype: {dtype}")

        # KL損失
        self.kl_weight = kl_loss_weight

        # 知覚損失
        self.perceptual_loss_fn = LPIPS().eval().to(device, dtype)
        self.perceptual_loss_fn.requires_grad_(False)
        self.perceptual_loss_weight = perceptual_loss_weight

        # 再構成損失とKL損失のバランスを取るための対数分散パラメータ
        self.logvar = nn.Parameter(torch.ones(size=()) * logvar_init)

    def forward(
        self,
        video,
        recon_video,
        posterior,
    ) -> dict:
        """
        VAEの損失計算を実行する関数

        Args:
            video (torch.Tensor):
                元の動画テンソル、形状は (B, C, T, H, W)
            recon_video (torch.Tensor):
                再構成された動画テンソル、形状は (B, C, T, H, W)
            posterior (Distribution | None):
                潜在変数の確率分布、KL損失の計算に使用
                Noneの場合、KL損失は0になる
        Returns:
            dict:
                各損失成分を含む辞書
                - "nll_loss": 負の対数尤度損失
                - "kl_loss": KL損失
                - "recon_loss": 再構成損失
                - "perceptual_loss": 知覚損失
        """
        log_.info(f"VAELoss forward: {video.shape=} {recon_video.shape=}")


        # 1) 動画を大量の画像に変換

        video.size(0)

        # (B, C, T, H, W) -> (B*T, C, H, W)
        video = rearrange(video, "b c t h w -> (b t) c h w").contiguous()

        # (B, C, T, H, W) -> (B*T, C, H, W)
        recon_video = rearrange(recon_video, "b c t h w -> (b t) c h w").contiguous()

        # 2) 再構成損失を計算

        recon_loss = l1(video, recon_video)

        # 3) 知覚損失を計算

        perceptual_loss = self.perceptual_loss_fn(video, recon_video)

        # 4) NLL損失を計算

        # NLL = 再構成損失 + 知覚損失 * 重み
        nll_loss = recon_loss + perceptual_loss * self.perceptual_loss_weight

        # 学習可能な対数分散パラメータを使用してNLL損失を調整
        # KL損失と再構成損失のバランスを自動的に取れる
        nll_loss = nll_loss / torch.exp(self.logvar) + self.logvar

        # 5) バッチ平均を計算

        # NLL損失はバッチ内の各要素に対して平均を計算
        nll_loss = batch_mean(nll_loss)

        # 再構成損失はバッチ内の各要素に対して平均を計算
        recon_loss = batch_mean(recon_loss)

        # 知覚損失はバッチ内の各要素に対して平均を計算し、元の動画の要素数でスケーリング
        numel_elements = video.numel() // video.size(0)
        perceptual_loss = batch_mean(perceptual_loss) * numel_elements

        # 6) KL損失を計算

        if posterior is None:
            kl_loss = torch.tensor(0.0).to(video.device, video.dtype)
        else:
            # KL損失を計算
            # 潜在変数の確率分布が標準正規分布からどれだけ歪んでいるか
            kl_loss = posterior.kl()

            kl_loss = batch_mean(kl_loss)

        # KL損失に重みを適用
        weighted_kl_loss = kl_loss * self.kl_weight

        return {
            "nll_loss": nll_loss,
            "kl_loss": weighted_kl_loss,
            "recon_loss": recon_loss,
            "perceptual_loss": perceptual_loss,
        }

vae_loss_fn = VAELoss(**cfg.vae_loss_config, device=device, dtype=dtype)

### EMAモデルの構築

EMA = Exponential Moving Average（指数移動平均）

訓練中のモデルの移動平均をとった汎化性能が相対的に高いモデルで、評価や推論に使用する

In [ ]:
def record_model_param_shape(model: torch.nn.Module) -> dict:
    """
    モデルパラメータの形状を記録する関数
    モデルの状態を保存するときに使用される

    Args:
        model (torch.nn.Module): 形状を記録するモデル
    Returns:
        dict: モデルパラメータの形状
    """

    param_shape = {}

    for name, param in model.named_parameters():
        # パラメータの形状を記録
        param_shape[name] = param.shape

    return param_shape

In [ ]:

# EMAモデルが有効な場合
# True
if cfg.get("ema_decay", None) is not None:
    # メインのモデルをコピーして、EMAモデルを作成
    ema = deepcopy(model).cpu().eval().requires_grad_(False)

    # EMAモデルのパラメータ形状を記録
    ema_shape_dict = record_model_param_shape(ema)
    logger.info("EMA model created.")
else:
    ema = ema_shape_dict = None
    logger.info("No EMA model created.")

In [ ]:
@torch.no_grad()
def update_ema(
    ema_model: torch.nn.Module, model: torch.nn.Module, optimizer=None, decay: float = 0.9999, sharded: bool = True
):
    """
    EMAモデルを現在のモデルに向けて更新する関数

    Args:
        ema_model (torch.nn.Module): EMAモデル
        model (torch.nn.Module): 現在のモデル
        optimizer (torch.optim.Optimizer): オプティマイザ
        decay (float): 減衰率
        sharded (bool): モデルがシャーディングされているかどうか
    """
    ema_params = OrderedDict(ema_model.named_parameters())
    model_params = OrderedDict(model.named_parameters())

    for name, param in model_params.items():
        if name == "pos_embed":
            continue
        if not param.requires_grad:
            continue
        if not sharded:
            param_data = param.data
            ema_params[name].mul_(decay).add_(param_data, alpha=1 - decay)
        else:
            if param.data.dtype != torch.float32:
                param_id = id(param)
                master_param = optimizer.get_working_to_master_map()[param_id]
                param_data = master_param.data
            else:
                param_data = param.data
            ema_params[name].mul_(decay).add_(param_data, alpha=1 - decay)

### ディスクリミネーターモデルの作成

DC-AEの第2ステップで使用

In [ ]:
use_discriminator = cfg.get("discriminator", None) is not None
use_discriminator

In [ ]:
# False
if use_discriminator:
    discriminator = build_module(cfg.discriminator, MODELS).to(device, dtype).train()
    log_model_params(discriminator)
    generator_loss_fn = GeneratorLoss(**cfg.gen_loss_config)
    discriminator_loss_fn = DiscriminatorLoss(**cfg.disc_loss_config)

### オプティマイザの作成

In [ ]:
cfg.optim

In [ ]:
optimizer = create_optimizer(model, cfg.optim)
optimizer

### 学習率スケジューラの作成

In [ ]:
# エポックあたりのステップ数を計算
num_steps_per_epoch = len(dataloader)
num_steps_per_epoch

In [ ]:
cfg.lr_scheduler

In [ ]:
lr_scheduler = create_lr_scheduler(
    optimizer=optimizer,
    num_steps_per_epoch=num_steps_per_epoch,
    epochs=cfg.get("epochs", 1000),
    **cfg.lr_scheduler
)
lr_scheduler

### ディスクリミネーターオプティマイザ

In [ ]:
# False
if use_discriminator:
    disc_optimizer = create_optimizer(discriminator, cfg.optim_discriminator)
    disc_lr_scheduler = create_lr_scheduler(
        optimizer=disc_optimizer,
        num_steps_per_epoch=num_steps_per_epoch,
        epochs=cfg.get("epochs", 1000),
        **cfg.disc_lr_scheduler,
    )

### 訓練の準備

In [ ]:
# デフォルトのデータ型を設定
torch.set_default_dtype(dtype)
dtype

In [ ]:
# Boosterを使用して、分散訓練に対応させる
model, optimizer, _, dataloader, lr_scheduler = booster.boost(
    model=model,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    dataloader=dataloader,
)

In [ ]:
# False
if use_discriminator:
    discriminator, disc_optimizer, _, _, disc_lr_scheduler = booster.boost(
        model=discriminator,
        optimizer=disc_optimizer,
        lr_scheduler=disc_lr_scheduler,
    )

In [ ]:
# torch.set_default_dtype(torch.float)
# logger.info("Boosted model for distributed training")

In [ ]:
# グローバル変数を設定

# エポック数
cfg_epochs = cfg.get("epochs", 1000)

# 動画と画像の混合戦略
# mixed_video_image
mixed_strategy = cfg.get("mixed_strategy", None)

# 動画に対して画像を混合する比率
# 0.2
mixed_image_ratio = cfg.get("mixed_image_ratio", 0.0)

# 分散訓練のランク数を取得
num_ranks = dist.get_world_size()
print("num_ranks:", num_ranks)

# 混合画像比率をランク数に基づいて調整
# 複数ランクの場合、各ランクが異なる画像を処理できるように調整
# 0.2
modulated_mixed_image_ratio = (
    num_ranks * mixed_image_ratio / (num_ranks - 1) if num_ranks > 1 else mixed_image_ratio
)

if is_log_process(plugin_type, plugin_config):
    print("modulated mixed image ratio:", modulated_mixed_image_ratio)

# エポック数、ステップ数、ログステップ数、累積ステップ数の初期化
start_epoch = start_step = log_step = acc_step = 0

# 1エポックあたりの累積損失
running_loss = dict(
    all=0.0, # 全損失の合計
    nll=0.0, # 負の対数尤度損失
    nll_rec=0.0, # 再構成損失
    nll_per=0.0, # 知覚損失
    kl=0.0, # KL損失
    gen=0.0, # ジェネレータ損失
    gen_w=0.0, # 重み付きジェネレータ損失
    disc=0.0, # 識別器損失
    debug=0.0, # デバッグ用損失
)

In [ ]:
def all_reduce_sum(tensor: torch.Tensor) -> torch.Tensor:
    """
    全てのプロセスでテンソルを合計する関数
    log_loss関数で使用される
    """
    dist.all_reduce(tensor=tensor, group=get_data_parallel_group())
    return tensor

In [ ]:
def log_loss(name, loss, loss_dict, use_video):
    """
    損失をログに記録する関数
    Args:
        name (str): 損失の名前
        loss (torch.Tensor): 損失値のテンソル
        loss_dict (dict): 損失値を格納する辞書
        use_video (int): 動画データが使用されたかどうかのフラグ（0または1）
    """
    # 動画を使用しない場合、損失を0に設定
    if use_video == 0:
        loss.data = torch.tensor(0.0, device=device, dtype=dtype)

    # 全てのプロセスで損失を合計
    all_reduce_sum(loss.data)

    # 動画を使用したプロセス数を合計
    num_video = torch.tensor(use_video, device=device, dtype=dtype)
    all_reduce_sum(num_video)

    # 動画を扱ったプロセス数で損失の平均を計算
    loss_item = loss.item() / num_video.item()

    # 辞書に損失を記録
    loss_dict[name] = loss_item

    # 累積損失を更新
    running_loss[name] += loss_item

logger.info("Training for %s epochs with %s steps per epoch", cfg_epochs, num_steps_per_epoch)

### EMAモデルをシャーディング

In [ ]:
def model_sharding(model: torch.nn.Module, device: torch.device = None):
    """
    複数GPUにわたってモデルパラメータを分割する関数

    Args:
        model (torch.nn.Module): 分割するモデル
        device (torch.device): モデルを分割するデバイス
    """
    log_.info(f"model_sharding {device=}")

    # 1) グローバルランクとワールドサイズを取得

    global_rank = dist.get_rank()
    world_size = dist.get_world_size()

    # 2) 各パラメータごとに分割を実行
    for _, param in model.named_parameters():

        # パラメータのデバイスを設定
        if device is None:
            device = param.device

        # パラメータの要素数に基づいてパディングサイズを計算
        padding_size = (world_size - param.numel() % world_size) % world_size
        log_.debug(f"{param.shape=} {param.numel()=} {padding_size=}")

        if padding_size > 0:
            padding_param = torch.nn.functional.pad(param.data.view(-1), [0, padding_size])
        else:
            padding_param = param.data.view(-1)

        # パラメータを均等に分割

        splited_params = padding_param.split(padding_param.numel() // world_size)

        splited_params = splited_params[global_rank]

        # デバイスへの転送

        param.data = splited_params.to(device)

if ema is not None:
    model_sharding(ema)
    ema = ema.to(device)

In [ ]:
# False
if cfg.get("freeze_layers", None) == "all":
    for param in model.module.parameters():
        param.requires_grad = False
    print("all layers frozen")

### 訓練の再開

In [ ]:
# False
if cfg.get("load", None) is not None:
    logger.info("Loading checkpoint from %s", cfg.load)
    start_epoch = cfg.get("start_epoch", None)
    start_step = cfg.get("start_step", None)
    ret = checkpoint_io.load(
        booster,
        cfg.load,
        model=model,
        ema=ema,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        sampler=(
            None if start_step is not None else sampler
        ),  # if specify start step, set last_micro_batch_access_index of a new sampler instead
    )
    if start_step is not None:
        # if start step exceeds data length, go to next epoch
        if start_step > num_steps_per_epoch:
            start_epoch = (
                start_epoch + start_step // num_steps_per_epoch
                if start_epoch is not None
                else start_step // num_steps_per_epoch
            )
            start_step = start_step % num_steps_per_epoch
        sampler.set_step(start_step)

    start_epoch = start_epoch if start_epoch is not None else ret[0]
    start_step = start_step if start_step is not None else ret[1]

    if (
        use_discriminator
        and os.path.exists(os.path.join(cfg.load, "discriminator"))
        and not cfg.get("restart_disc", False)
    ):
        booster.load_model(discriminator, os.path.join(cfg.load, "discriminator"))
        if cfg.get("load_optimizer", True):
            booster.load_optimizer(disc_optimizer, os.path.join(cfg.load, "disc_optimizer"))
            if disc_lr_scheduler is not None:
                booster.load_lr_scheduler(disc_lr_scheduler, os.path.join(cfg.load, "disc_lr_scheduler"))
            if cfg.get("disc_lr", None) is not None:
                set_lr(disc_optimizer, disc_lr_scheduler, cfg.disc_lr)

    logger.info("Loaded checkpoint %s at epoch %s step %s", cfg.load, start_epoch, start_step)

    if cfg.get("lr", None) is not None:
        set_lr(optimizer, lr_scheduler, cfg.lr, cfg.get("initial_lr", None))

    if cfg.get("update_warmup_steps", False):
        assert (
            cfg.lr_scheduler.get("warmup_steps", None) is not None
        ), "you need to set lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
        set_warmup_steps(lr_scheduler, cfg.lr_scheduler.warmup_steps)
        if use_discriminator:
            assert (
                cfg.disc_lr_scheduler.get("warmup_steps", None) is not None
            ), "you need to set disc_lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
            set_warmup_steps(disc_lr_scheduler, cfg.disc_lr_scheduler.warmup_steps)

### 訓練ループ

In [ ]:
# データローダとサンプラーを際初期化
dataloader, sampler = prepare_dataloader(
    bucket_config=cfg.get("bucket_config", None),
    num_bucket_build_workers=cfg.get("num_bucket_build_workers", 1),
    **dataloader_args,
)
dataloader, sampler

In [ ]:
# 分散訓練の同期を待機
dist.barrier()

# 勾配累積ステップ数を取得
accumulation_steps = int(cfg.get("accumulation_steps", 1))
logger.info("Using gradient accumulation steps: %s", accumulation_steps)

In [ ]:
for epoch in range(start_epoch, cfg_epochs):

    # 1) エポックの初期化

    # サンプラーのエポックを設定
    sampler.set_epoch(epoch)

    # データローダのイテレータを作成
    dataiter = iter(dataloader)

    logger.info("Beginning epoch %s...", epoch)

    # それぞれのランクで異なる乱数シードを設定
    random.seed(1024 + dist.get_rank())

    # 2) エポック内のループを開始

    with tqdm(
        enumerate(dataiter, start=start_step),
        desc=f"Epoch {epoch}",
        disable=not coordinator.is_master(),
        total=num_steps_per_epoch,
        initial=start_step,
    ) as pbar:

        # 2-1) バッチを取得

        pbar_iter = iter(pbar)

        def fetch_data():
            "非同期にデータを取得し、デバイスに転送する"
            # データローダからバッチを取得
            step, batch = next(pbar_iter)
            # ピン留めされたビデオテンソルを取得
            pinned_video = batch["video"]
            # ビデオテンソルをデバイスに転送（non_blocking=Trueで非同期転送）
            batch["video"] = pinned_video.to(device, dtype, non_blocking=True)
            return batch, step, pinned_video

        # 最初のバッチを先読み
        batch_, step_, pinned_video_ = fetch_data()

        # プロファイリングコンテキストを作成
        # torch.profilerを使用してパフォーマンスを測定する
        profiler_ctxt = (
            profile(
                activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                schedule=my_schedule,
                on_trace_ready=torch.profiler.tensorboard_trace_handler("./log/profile"),
                record_shapes=True,
                profile_memory=True,
                with_stack=True,
            )
            if cfg.get("profile", False)
            else nullcontext()
        )

        # パフォーマンスプロファイリングを開始
        with profiler_ctxt:

            # 3) ステップごとのループを開始

            for _ in range(start_step, num_steps_per_epoch):
                if cfg.get("profile", False) and _ == WARMUP + ACTIVE + WAIT + 3:
                    break

                # 3-1) 次のバッチを先読み

                batch, step, pinned_video = batch_, step_, pinned_video_

                if step + 1 < num_steps_per_epoch:
                    batch_, step_, pinned_video_ = fetch_data()

                # 3-2) ステップを更新

                global_step = epoch * num_steps_per_epoch + step

                actual_update_step = (global_step + 1) // accumulation_steps

                log_step += 1
                acc_step += 1

                # 3-3) 動画と画像の混合戦略を適用

                x = batch["video"]
                t_length = x.size(2)
                use_video = 1

                # True
                if mixed_strategy == "mixed_video_image":
                    # 20%の確率で画像のみを使用
                    if random.random() < modulated_mixed_image_ratio and dist.get_rank() != 0:
                        # NOTE: enable the first rank to use video
                        t_length = 1
                        use_video = 0
                # False
                elif mixed_strategy == "mixed_video_random":
                    t_length = random.randint(1, x.size(2))

                # 必要に応じて動画を画像に変換
                x = x[:, :, :t_length, :, :]

                # タイマーコンテキストを開始
                with Timer("model", log=True) if cfg.get("profile", False) else nullcontext():

                    # 3-4) モデルの順伝搬を実行

                    x_rec, posterior, z = model(x)

                    # プロファイリングステップを進める
                    if cfg.get("profile", False):
                        profiler_ctxt.step()

                    # ピン留めされたメモリを解放
                    if cache_pin_memory:
                        dataiter.remove_cache(pinned_video)

                    # 3-5) すべての損失を初期化

                    vae_loss = torch.tensor(0.0, device=device, dtype=dtype)
                    loss_dict = {}  # loss at every step

                    # 3-6) 損失を計算

                    ret = vae_loss_fn(x, x_rec, posterior)
                    nll_loss = ret["nll_loss"]
                    kl_loss = ret["kl_loss"]
                    recon_loss = ret["recon_loss"]
                    perceptual_loss = ret["perceptual_loss"]

                    # 総VAE損失を計算
                    vae_loss += nll_loss + kl_loss

                    # GANのジェネレータ損失を計算
                    # False
                    if use_discriminator:
                        # turn off grad update for disc
                        discriminator.requires_grad_(False)
                        fake_logits = discriminator(x_rec.contiguous())

                        generator_loss, g_loss = generator_loss_fn(
                            fake_logits,
                            nll_loss,
                            model.module.get_last_layer(),
                            actual_update_step,
                            is_training=model.training,
                        )

                        vae_loss += generator_loss
                        # turn on disc training
                        discriminator.requires_grad_(True)

                    # 3-7) 逆伝搬を実行

                    # 勾配の同期を制御するコンテキストを設定
                    ctx = (
                        booster.no_sync(model, optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )

                    # タイマーコンテキストを開始
                    with Timer("backward", log=True) if cfg.get("profile", False) else nullcontext():
                        with ctx:
                            booster.backward(loss=vae_loss / accumulation_steps, optimizer=optimizer)

                    # 3-8) パラメータを更新

                    # タイマーコンテキストを開始
                    with Timer("optimizer", log=True) if cfg.get("profile", False) else nullcontext():

                        if (step + 1) % accumulation_steps == 0:
                            optimizer.step()
                            optimizer.zero_grad()
                            if lr_scheduler is not None:
                                lr_scheduler.step(
                                    actual_update_step,
                                )

                            # 3-9) EMAモデルを更新

                            if ema is not None:
                                update_ema(
                                    ema,
                                    model.unwrap(),
                                    optimizer=optimizer,
                                    decay=cfg.get("ema_decay", 0.9999),
                                )

                # 3-10) 損失をログに記録

                log_loss("all", vae_loss, loss_dict, use_video)
                log_loss("nll", nll_loss, loss_dict, use_video)
                log_loss("nll_rec", recon_loss, loss_dict, use_video)
                log_loss("nll_per", perceptual_loss, loss_dict, use_video)
                log_loss("kl", kl_loss, loss_dict, use_video)
                if use_discriminator:
                    log_loss("gen_w", generator_loss, loss_dict, use_video)
                    log_loss("gen", g_loss, loss_dict, use_video)

                # 3-11) 識別器の損失の計算と更新

                # False
                if use_discriminator:
                    real_logits = discriminator(x.detach().contiguous())
                    fake_logits = discriminator(x_rec.detach().contiguous())
                    disc_loss = discriminator_loss_fn(
                        real_logits,
                        fake_logits,
                        actual_update_step,
                    )

                    # 識別器の逆伝播と更新

                    ctx = (
                        booster.no_sync(discriminator, disc_optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )
                    with ctx:
                        booster.backward(loss=disc_loss / accumulation_steps, optimizer=disc_optimizer)
                    if (step + 1) % accumulation_steps == 0:
                        disc_optimizer.step()
                        disc_optimizer.zero_grad()
                        if disc_lr_scheduler is not None:
                            disc_lr_scheduler.step(actual_update_step)

                    # 識別器の損失をログに記録
                    log_loss("disc", disc_loss, loss_dict, use_video)


                if (global_step + 1) % accumulation_steps == 0:

                    # 3-12) tensorboardとwandbへのログ記録

                    if coordinator.is_master() and actual_update_step % cfg.get("log_every", 1) == 0:
                        avg_loss = {k: v / log_step for k, v in running_loss.items()}
                        # progress bar
                        pbar.set_postfix(
                            {
                                # "step": step,
                                # "global_step": global_step,
                                # "actual_update_step": actual_update_step,
                                # "lr": optimizer.param_groups[0]["lr"],
                                **{k: f"{v:.2f}" for k, v in avg_loss.items()},
                            }
                        )
                        # tensorboard
                        tb_writer.add_scalar("loss", vae_loss.item(), actual_update_step)
                        # wandb
                        if cfg.get("wandb", False):
                            wandb.log(
                                {
                                    "iter": global_step,
                                    "epoch": epoch,
                                    "lr": optimizer.param_groups[0]["lr"],
                                    "avg_loss_": avg_loss,
                                    "avg_loss": avg_loss["all"],
                                    "loss_": loss_dict,
                                    "loss": vae_loss.item(),
                                    "global_grad_norm": optimizer.get_grad_norm(),
                                },
                                step=actual_update_step,
                            )

                        running_loss = {k: 0.0 for k in running_loss}
                        log_step = 0

                    # 3-13) チェックポイントの保存

                    ckpt_every = cfg.get("ckpt_every", 0)
                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0 and coordinator.is_master():
                        subprocess.run("sudo drop_cache", shell=True)

                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0:
                        # mannually garbage collection
                        gc.collect()

                        save_dir = checkpoint_io.save(
                            booster,
                            exp_dir,
                            model=model,
                            ema=ema,
                            optimizer=optimizer,
                            lr_scheduler=lr_scheduler,
                            sampler=sampler,
                            epoch=epoch,
                            step=step + 1,
                            global_step=global_step + 1,
                            batch_size=cfg.get("batch_size", None),
                            actual_update_step=actual_update_step,
                            ema_shape_dict=ema_shape_dict,
                            async_io=True,
                        )

                        if is_log_process(plugin_type, plugin_config):
                            os.system(f"chgrp -R share {save_dir}")

                        if use_discriminator:
                            booster.save_model(discriminator, os.path.join(save_dir, "discriminator"), shard=True)
                            booster.save_optimizer(
                                disc_optimizer,
                                os.path.join(save_dir, "disc_optimizer"),
                                shard=True,
                                size_per_shard=4096,
                            )
                            if disc_lr_scheduler is not None:
                                booster.save_lr_scheduler(
                                    disc_lr_scheduler, os.path.join(save_dir, "disc_lr_scheduler")
                                )
                        dist.barrier()

                        logger.info(
                            "Saved checkpoint at epoch %s, step %s, global_step %s to %s",
                            epoch,
                            step + 1,
                            actual_update_step,
                            save_dir,
                        )

                        # remove old checkpoints
                        rm_checkpoints(exp_dir, keep_n_latest=cfg.get("keep_n_latest", -1))
                        logger.info(
                            "Removed old checkpoints and kept %s latest ones.", cfg.get("keep_n_latest", -1)
                        )

        if cfg.get("profile", False):
            profiler_ctxt.export_chrome_trace("./log/profile/trace.json")

    sampler.reset()
    start_step = 0